In [ ]:
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
labels = np.load('/content/drive/My Drive/Colab Notebooks/labels.npy')
audios = np.load('/content/drive/My Drive/Colab Notebooks/audios.npy')

In [ ]:
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder, label_binarize
from sklearn.model_selection import train_test_split


In [ ]:
A_train, A_test, y_train, y_test = train_test_split(audios, labels, test_size=0.25, random_state=1)

In [ ]:
from keras.models import Sequential, Model
from keras.layers.core import Dense, Activation
from keras.layers import LSTM, Input, Flatten, Embedding, Conv1D, Conv2D, Dropout, MaxPooling2D

import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Flatten, GlobalMaxPooling1D
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score, f1_score

from keras.regularizers import l1, l2

In [ ]:
yt = label_binarize(y_train, classes=['angry', 'happy', 'sad', 'neutral', 'other', 'X'])
yt2 = label_binarize(y_test, classes=['angry', 'happy', 'sad', 'neutral', 'other', 'X'])

In [ ]:
A_train.shape

(954, 338368)

In [ ]:
#specs = np.load('/content/drive/My Drive/Colab Notebooks/specs.npy')
#S_train, S_test, = train_test_split(specs, test_size=0.25, random_state=1)

In [ ]:
A_train=A_train.reshape(A_train.shape[0],2*2*136,int(338368/272/2))
A_test = A_test.reshape(A_test.shape[0], 2*2*136,int(338368/272/2))
#A_train=A_train.reshape(A_train.shape[0],1,A_train.shape[1])
A_test.shape

(318, 544, 622)

In [ ]:
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Flatten, GlobalMaxPooling1D

# Define the LSTM model
model = Sequential()

# First LSTM layer with dropout
model.add(LSTM(512, return_sequences=True, input_shape=(A_train.shape[1], A_train.shape[2])))
model.add(Dropout(0.5))

# Second LSTM layer with dropout
model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.5))

# Reshape the output to 3D
#model.add(Reshape((1, 128)))

# Add a 1D global max pooling layer to reduce the dimensions
model.add(GlobalMaxPooling1D())

# Add a dense layer with activation function
model.add(Dense(64, activation='relu'))

# Add the final dense layer for prediction
model.add(Dense(6, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(A_train, yt, epochs=15, batch_size=32, shuffle=True, validation_split=0.2)

Epoch 1/15
24/24 [==============================] - 6s 164ms/step - loss: 1.4949 - accuracy: 0.3840 - val_loss: 1.4586 - val_accuracy: 0.4136
Epoch 2/15
24/24 [==============================] - 3s 132ms/step - loss: 1.3356 - accuracy: 0.4404 - val_loss: 1.4548 - val_accuracy: 0.4346
Epoch 3/15
24/24 [==============================] - 3s 133ms/step - loss: 1.2494 - accuracy: 0.5308 - val_loss: 1.4351 - val_accuracy: 0.4188
Epoch 4/15
24/24 [==============================] - 3s 133ms/step - loss: 1.1328 - accuracy: 0.5872 - val_loss: 1.4057 - val_accuracy: 0.4084
Epoch 5/15
24/24 [==============================] - 3s 135ms/step - loss: 0.9777 - accuracy: 0.6697 - val_loss: 1.4626 - val_accuracy: 0.4136
Epoch 6/15
24/24 [==============================] - 3s 134ms/step - loss: 0.8226 - accuracy: 0.7261 - val_loss: 1.4278 - val_accuracy: 0.3822
Epoch 7/15
24/24 [==============================] - 3s 135ms/step - loss: 0.6449 - accuracy: 0.8060 - val_loss: 1.5411 - val_accuracy: 0.3613
Epoch 

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score, f1_score

y_pred = model.predict(A_test)
y_pred = np.argmax(y_pred, axis=1)
y_arg = np.argmax(yt2, axis=1)

test_loss, test_acc = model.evaluate(A_test, yt2)

print('Test accuracy:', test_acc)

f1 = f1_score(y_arg, y_pred, average='weighted')
print("F1-score:", f1)

print(confusion_matrix(y_arg, y_pred))

10/10 [==============================] - 1s 56ms/step - loss: 1.9639 - accuracy: 0.3553
Test accuracy: 0.3553459048271179
F1-score: 0.33817482247487357
[[ 8 11 11 23  0 20]
 [ 4  4  5 21  0 18]
 [ 2  4 11 26  0  4]
 [ 4  8 19 85  0 16]
 [ 0  1  0  0  0  0]
 [ 1  1  2  4  0  5]]


In [ ]:
### PARAMETER TUNING (manual)

#1 - LSTM layers - 1/2/3 (default was 2)

# 1 layer:

# Define the LSTM model
model = Sequential()

# First LSTM layer with dropout
model.add(LSTM(512, return_sequences=True, input_shape=(A_train.shape[1], A_train.shape[2])))
model.add(Dropout(0.5))

# Reshape the output to 3D
#model.add(Reshape((1, 128)))

# Add a 1D global max pooling layer to reduce the dimensions
model.add(GlobalMaxPooling1D())

# Add a dense layer with activation function
model.add(Dense(64, activation='sigmoid'))

# Add the final dense layer for prediction
model.add(Dense(6, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(A_train, yt, epochs=15, batch_size=32, shuffle=True, validation_split=0.2)

Epoch 1/15
24/24 [==============================] - 5s 132ms/step - loss: 1.8990 - accuracy: 0.3198 - val_loss: 1.4970 - val_accuracy: 0.3927
Epoch 2/15
24/24 [==============================] - 3s 108ms/step - loss: 1.4339 - accuracy: 0.4089 - val_loss: 1.4947 - val_accuracy: 0.3770
Epoch 3/15
24/24 [==============================] - 3s 107ms/step - loss: 1.3271 - accuracy: 0.4548 - val_loss: 1.4536 - val_accuracy: 0.4241
Epoch 4/15
24/24 [==============================] - 3s 108ms/step - loss: 1.2533 - accuracy: 0.5125 - val_loss: 1.4752 - val_accuracy: 0.3560
Epoch 5/15
24/24 [==============================] - 3s 109ms/step - loss: 1.2728 - accuracy: 0.4758 - val_loss: 1.4644 - val_accuracy: 0.4031
Epoch 6/15
24/24 [==============================] - 3s 112ms/step - loss: 1.1518 - accuracy: 0.5413 - val_loss: 1.4562 - val_accuracy: 0.4136
Epoch 7/15
24/24 [==============================] - 3s 111ms/step - loss: 1.1990 - accuracy: 0.5400 - val_loss: 1.4720 - val_accuracy: 0.4188
Epoch 

# Base

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score, f1_score

y_pred = model.predict(A_test)
y_pred = np.argmax(y_pred, axis=1)
y_arg = np.argmax(yt2, axis=1)

test_loss, test_acc = model.evaluate(A_test, yt2)

print('Test accuracy:', test_acc)

f1 = f1_score(y_arg, y_pred, average='weighted')
print("F1-score:", f1)

print(confusion_matrix(y_arg, y_pred))

10/10 [==============================] - 0s 44ms/step - loss: 1.4030 - accuracy: 0.4654
Test accuracy: 0.4654088020324707
F1-score: 0.3844911675942661
[[57  4  0 12  0  0]
 [27  3  0 22  0  0]
 [14  1  0 32  0  0]
 [43  1  0 88  0  0]
 [ 1  0  0  0  0  0]
 [ 8  0  0  5  0  0]]


In [ ]:
### 3 LSTM layers:

import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Flatten, GlobalMaxPooling1D, Reshape

# Define the LSTM model
model = Sequential()

# First LSTM layer with dropout
model.add(LSTM(512, return_sequences=True, input_shape=(A_train.shape[1], A_train.shape[2])))
model.add(Dropout(0.5))

# Second LSTM layer with dropout
model.add(LSTM(256, return_sequences=True))
model.add(Dropout(0.5))

model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.5))

# Reshape the output to 3D
#model.add(Reshape((1, 128)))

# Add a 1D global max pooling layer to reduce the dimensions
model.add(GlobalMaxPooling1D())

# Add a dense layer with activation function
model.add(Dense(64, activation='relu'))

# Add the final dense layer for prediction
model.add(Dense(6, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(A_train, yt, epochs=15, batch_size=32, shuffle=True, validation_split=0.2)

Epoch 1/15
24/24 [==============================] - 9s 212ms/step - loss: 1.4954 - accuracy: 0.3893 - val_loss: 1.4900 - val_accuracy: 0.4188
Epoch 2/15
24/24 [==============================] - 4s 173ms/step - loss: 1.3270 - accuracy: 0.4391 - val_loss: 1.4572 - val_accuracy: 0.3927
Epoch 3/15
24/24 [==============================] - 4s 172ms/step - loss: 1.1526 - accuracy: 0.5688 - val_loss: 1.5196 - val_accuracy: 0.3508
Epoch 4/15
24/24 [==============================] - 4s 173ms/step - loss: 0.9349 - accuracy: 0.6632 - val_loss: 1.4792 - val_accuracy: 0.3874
Epoch 5/15
24/24 [==============================] - 4s 174ms/step - loss: 0.6502 - accuracy: 0.7772 - val_loss: 1.5565 - val_accuracy: 0.3403
Epoch 6/15
24/24 [==============================] - 4s 175ms/step - loss: 0.5095 - accuracy: 0.8283 - val_loss: 1.5592 - val_accuracy: 0.3770
Epoch 7/15
24/24 [==============================] - 4s 176ms/step - loss: 0.4399 - accuracy: 0.8519 - val_loss: 1.5861 - val_accuracy: 0.3927
Epoch 

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score, f1_score

y_pred = model.predict(A_test)
y_pred = np.argmax(y_pred, axis=1)
y_arg = np.argmax(yt2, axis=1)

test_loss, test_acc = model.evaluate(A_test, yt2)

print('Test accuracy:', test_acc)

f1 = f1_score(y_arg, y_pred, average='weighted')
print("F1-score:", f1)

print(confusion_matrix(y_arg, y_pred))

10/10 [==============================] - 1s 70ms/step - loss: 1.6617 - accuracy: 0.3994
Test accuracy: 0.39937105774879456
F1-score: 0.3696581197670411
[[15 20  9 29  0  0]
 [10 18  2 21  0  1]
 [ 7  5  6 28  0  1]
 [15 25  4 88  0  0]
 [ 0  1  0  0  0  0]
 [ 1  5  1  6  0  0]]


In [ ]:
## 1 Layer has much better accuracy, so I will use that from now on when testing other parameters, as the amount of layers changes the model a lot.

# Regularization



In [ ]:
### Regularization testing

In [ ]:
### L2(0.001)

from keras.regularizers import l1, l2

# Define the LSTM model
model = Sequential()

# First LSTM layer with dropout
model.add(LSTM(512, return_sequences=True, input_shape=(A_train.shape[1], A_train.shape[2]), activity_regularizer=l2(0.001)))
model.add(Dropout(0.5))

# Reshape the output to 3D
#model.add(Reshape((1, 128)))

# Add a 1D global max pooling layer to reduce the dimensions
model.add(GlobalMaxPooling1D())

# Add a dense layer with activation function
model.add(Dense(64, activation='relu'))

# Add the final dense layer for prediction
model.add(Dense(6, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(A_train, yt, epochs=15, batch_size=32, shuffle=True, validation_split=0.2)

Epoch 1/15
24/24 [==============================] - 5s 148ms/step - loss: 9.0606 - accuracy: 0.3486 - val_loss: 7.3592 - val_accuracy: 0.2775
Epoch 2/15
24/24 [==============================] - 3s 113ms/step - loss: 6.3853 - accuracy: 0.3906 - val_loss: 6.0987 - val_accuracy: 0.3822
Epoch 3/15
24/24 [==============================] - 3s 112ms/step - loss: 5.6449 - accuracy: 0.4731 - val_loss: 5.8367 - val_accuracy: 0.3560
Epoch 4/15
24/24 [==============================] - 3s 112ms/step - loss: 5.6003 - accuracy: 0.4902 - val_loss: 6.0478 - val_accuracy: 0.3298
Epoch 5/15
24/24 [==============================] - 3s 113ms/step - loss: 5.5593 - accuracy: 0.4875 - val_loss: 5.8891 - val_accuracy: 0.3979
Epoch 6/15
24/24 [==============================] - 3s 113ms/step - loss: 5.3517 - accuracy: 0.5360 - val_loss: 5.8098 - val_accuracy: 0.3351
Epoch 7/15
24/24 [==============================] - 3s 113ms/step - loss: 5.1699 - accuracy: 0.5557 - val_loss: 5.6498 - val_accuracy: 0.3560
Epoch 

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score, f1_score


y_pred = model.predict(A_test)
y_pred = np.argmax(y_pred, axis=1)
y_arg = np.argmax(yt2, axis=1)

test_loss, test_acc = model.evaluate(A_test, yt2)

print('Test accuracy:', test_acc)

f1 = f1_score(y_arg, y_pred, average='micro')
print("F1-score:", f1)

print(confusion_matrix(y_arg, y_pred))

10/10 [==============================] - 0s 46ms/step - loss: 5.0755 - accuracy: 0.3208
Test accuracy: 0.3207547068595886
F1-score: 0.32075471698113206
[[36 24  2 11  0  0]
 [24 18  1  9  0  0]
 [10 20  1 16  0  0]
 [40 40  5 47  0  0]
 [ 1  0  0  0  0  0]
 [ 4  4  0  5  0  0]]


In [ ]:
# Not very good, lets try L1

from keras.regularizers import l1, l2

# Define the LSTM model
model = Sequential()

# First LSTM layer with dropout
model.add(LSTM(512, return_sequences=True, input_shape=(A_train.shape[1], A_train.shape[2]), activity_regularizer=l1(0.001)))
model.add(Dropout(0.5))

# Add a 1D global max pooling layer to reduce the dimensions
model.add(GlobalMaxPooling1D())

# Add a dense layer with activation function
model.add(Dense(64, activation='relu'))

# Add the final dense layer for prediction
model.add(Dense(6, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(A_train, yt, epochs=15, batch_size=32, shuffle=True, validation_split=0.2)


Epoch 1/15
24/24 [==============================] - 5s 136ms/step - loss: 13.5135 - accuracy: 0.3119 - val_loss: 11.1845 - val_accuracy: 0.3979
Epoch 2/15
24/24 [==============================] - 3s 111ms/step - loss: 9.7748 - accuracy: 0.4299 - val_loss: 9.5042 - val_accuracy: 0.3246
Epoch 3/15
24/24 [==============================] - 3s 112ms/step - loss: 8.5453 - accuracy: 0.4509 - val_loss: 8.7977 - val_accuracy: 0.3298
Epoch 4/15
24/24 [==============================] - 3s 112ms/step - loss: 7.8252 - accuracy: 0.5334 - val_loss: 8.2041 - val_accuracy: 0.3979
Epoch 5/15
24/24 [==============================] - 3s 113ms/step - loss: 7.2836 - accuracy: 0.5649 - val_loss: 7.8739 - val_accuracy: 0.3298
Epoch 6/15
24/24 [==============================] - 3s 112ms/step - loss: 6.9260 - accuracy: 0.6094 - val_loss: 7.5532 - val_accuracy: 0.4293
Epoch 7/15
24/24 [==============================] - 3s 112ms/step - loss: 6.6394 - accuracy: 0.6448 - val_loss: 7.3678 - val_accuracy: 0.3613
Epoc

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score, f1_score


y_pred = model.predict(A_test)
y_pred = np.argmax(y_pred, axis=1)
y_arg = np.argmax(yt2, axis=1)

test_loss, test_acc = model.evaluate(A_test, yt2)

print('Test accuracy:', test_acc)

f1 = f1_score(y_arg, y_pred, average='weighted')
print("F1-score:", f1)

print(confusion_matrix(y_arg, y_pred))

10/10 [==============================] - 0s 45ms/step - loss: 6.1788 - accuracy: 0.4528
Test accuracy: 0.4528301954269409
F1-score: 0.4528301886792453
[[41  7  0 25  0  0]
 [18  4  0 30  0  0]
 [12  5  1 29  0  0]
 [24  8  2 98  0  0]
 [ 1  0  0  0  0  0]
 [ 3  1  0  9  0  0]]


In [ ]:
from keras.regularizers import l1, l2

# Define the LSTM model
model = Sequential()

# First LSTM layer with dropout
model.add(LSTM(512, return_sequences=True, input_shape=(A_train.shape[1], A_train.shape[2]), activity_regularizer=l1(0.001)))
model.add(Dropout(0.5))

# Add a 1D global max pooling layer to reduce the dimensions
model.add(GlobalMaxPooling1D())

# Add a dense layer with activation function
model.add(Dense(64, activation='relu'))

# Add the final dense layer for prediction
model.add(Dense(6, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(A_train, yt, epochs=15, batch_size=32, shuffle=True, validation_split=0.2)

Epoch 1/15
24/24 [==============================] - 5s 138ms/step - loss: 13.4296 - accuracy: 0.3460 - val_loss: 11.1980 - val_accuracy: 0.3613
Epoch 2/15
24/24 [==============================] - 3s 116ms/step - loss: 9.8019 - accuracy: 0.4181 - val_loss: 9.5827 - val_accuracy: 0.2670
Epoch 3/15
24/24 [==============================] - 3s 116ms/step - loss: 8.5362 - accuracy: 0.4626 - val_loss: 8.7098 - val_accuracy: 0.4241
Epoch 4/15
24/24 [==============================] - 3s 116ms/step - loss: 7.7845 - accuracy: 0.5583 - val_loss: 8.2123 - val_accuracy: 0.4241
Epoch 5/15
24/24 [==============================] - 3s 116ms/step - loss: 7.3420 - accuracy: 0.5623 - val_loss: 7.8837 - val_accuracy: 0.3874
Epoch 6/15
24/24 [==============================] - 3s 116ms/step - loss: 6.9944 - accuracy: 0.5885 - val_loss: 7.6218 - val_accuracy: 0.3717
Epoch 7/15
24/24 [==============================] - 3s 121ms/step - loss: 6.6868 - accuracy: 0.6265 - val_loss: 7.4737 - val_accuracy: 0.3037
Epoc

In [ ]:
# Reevaluation

y_pred = model.predict(A_test)
y_pred = np.argmax(y_pred, axis=1)
y_arg = np.argmax(yt2, axis=1)

test_loss, test_acc = model.evaluate(A_test, yt2)

print('Test accuracy:', test_acc)

f1 = f1_score(y_arg, y_pred, average='weighted')
print("F1-score:", f1)

print(confusion_matrix(y_arg, y_pred))

10/10 [==============================] - 0s 46ms/step - loss: 6.2564 - accuracy: 0.2704
Test accuracy: 0.2704402506351471
F1-score: 0.23673014429958145
[[ 1 57  3 12  0  0]
 [ 2 37  0 13  0  0]
 [ 0 32  3 12  0  0]
 [ 7 72  8 45  0  0]
 [ 0  1  0  0  0  0]
 [ 0  6  1  6  0  0]]


In [ ]:
# L1 looks much better. Lets try and alter the values.

model = Sequential()

# First LSTM layer with dropout
model.add(LSTM(512, return_sequences=True, input_shape=(A_train.shape[1], A_train.shape[2]), activity_regularizer=l1(0.01)))
model.add(Dropout(0.5))

# Add a 1D global max pooling layer to reduce the dimensions
model.add(GlobalMaxPooling1D())

# Add a dense layer with activation function
model.add(Dense(64, activation='relu'))

# Add the final dense layer for prediction
model.add(Dense(6, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(A_train, yt, epochs=15, batch_size=32, shuffle=True, validation_split=0.2)

Epoch 1/15
24/24 [==============================] - 5s 137ms/step - loss: 95.4151 - accuracy: 0.3696 - val_loss: 64.7393 - val_accuracy: 0.4084
Epoch 2/15
24/24 [==============================] - 3s 112ms/step - loss: 50.6905 - accuracy: 0.4142 - val_loss: 47.3041 - val_accuracy: 0.3141
Epoch 3/15
24/24 [==============================] - 3s 113ms/step - loss: 38.9329 - accuracy: 0.4456 - val_loss: 39.1938 - val_accuracy: 0.3979
Epoch 4/15
24/24 [==============================] - 3s 113ms/step - loss: 32.8701 - accuracy: 0.4915 - val_loss: 34.5243 - val_accuracy: 0.4031
Epoch 5/15
24/24 [==============================] - 3s 112ms/step - loss: 29.1962 - accuracy: 0.5623 - val_loss: 31.5353 - val_accuracy: 0.3717
Epoch 6/15
24/24 [==============================] - 3s 113ms/step - loss: 26.7649 - accuracy: 0.5623 - val_loss: 29.6318 - val_accuracy: 0.4136
Epoch 7/15
24/24 [==============================] - 3s 113ms/step - loss: 25.0225 - accuracy: 0.6566 - val_loss: 27.7543 - val_accuracy:

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score, f1_score


y_pred = model.predict(A_test)
y_pred = np.argmax(y_pred, axis=1)
y_arg = np.argmax(yt2, axis=1)

test_loss, test_acc = model.evaluate(A_test, yt2)

print('Test accuracy:', test_acc)

f1 = f1_score(y_arg, y_pred, average='micro')
print("F1-score:", f1)

print(confusion_matrix(y_arg, y_pred))

10/10 [==============================] - 0s 45ms/step - loss: 20.4333 - accuracy: 0.4088
Test accuracy: 0.4088050425052643
F1-score: 0.4088050314465409
[[32 25  5 11  0  0]
 [13 14  4 21  0  0]
 [14  6  1 24  0  2]
 [22 16 11 83  0  0]
 [ 1  0  0  0  0  0]
 [ 3  4  1  5  0  0]]


In [ ]:
## Too large probably, lets try smaller?


model = Sequential()

# First LSTM layer with dropout
model.add(LSTM(512, return_sequences=True, input_shape=(A_train.shape[1], A_train.shape[2]), activity_regularizer=l1(0.0001)))
model.add(Dropout(0.5))

# Add a 1D global max pooling layer to reduce the dimensions
model.add(GlobalMaxPooling1D())

# Add a dense layer with activation function
model.add(Dense(64, activation='relu'))

# Add the final dense layer for prediction
model.add(Dense(6, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(A_train, yt, epochs=15, batch_size=32, shuffle=True, validation_split=0.2)

Epoch 1/15
24/24 [==============================] - 5s 156ms/step - loss: 3.3329 - accuracy: 0.3159 - val_loss: 2.9782 - val_accuracy: 0.3455
Epoch 2/15
24/24 [==============================] - 3s 115ms/step - loss: 2.8151 - accuracy: 0.3997 - val_loss: 2.8497 - val_accuracy: 0.4136
Epoch 3/15
24/24 [==============================] - 3s 115ms/step - loss: 2.6876 - accuracy: 0.4522 - val_loss: 2.8291 - val_accuracy: 0.4084
Epoch 4/15
24/24 [==============================] - 3s 114ms/step - loss: 2.5416 - accuracy: 0.5007 - val_loss: 2.7383 - val_accuracy: 0.4136
Epoch 5/15
24/24 [==============================] - 3s 115ms/step - loss: 2.4870 - accuracy: 0.4967 - val_loss: 2.7167 - val_accuracy: 0.4136
Epoch 6/15
24/24 [==============================] - 3s 114ms/step - loss: 2.3922 - accuracy: 0.5111 - val_loss: 2.6690 - val_accuracy: 0.4136
Epoch 7/15
24/24 [==============================] - 3s 115ms/step - loss: 2.3174 - accuracy: 0.5623 - val_loss: 2.6394 - val_accuracy: 0.4293
Epoch 

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score, f1_score


y_pred = model.predict(A_test)
y_pred = np.argmax(y_pred, axis=1)
y_arg = np.argmax(yt2, axis=1)

test_loss, test_acc = model.evaluate(A_test, yt2)

print('Test accuracy:', test_acc)

f1 = f1_score(y_arg, y_pred, average='micro')
print("F1-score:", f1)

print(confusion_matrix(y_arg, y_pred))

10/10 [==============================] - 0s 45ms/step - loss: 2.5416 - accuracy: 0.3836
Test accuracy: 0.3836477994918823
F1-score: 0.3836477987421384
[[ 5 43  0 25  0  0]
 [ 3 30  0 19  0  0]
 [ 0 18  0 29  0  0]
 [ 0 44  1 87  0  0]
 [ 1  0  0  0  0  0]
 [ 0  6  0  7  0  0]]


In [ ]:
# 0.001 was good for L1, lets see what value works best with L2

model = Sequential()

# First LSTM layer with dropout
model.add(LSTM(512, return_sequences=True, input_shape=(A_train.shape[1], A_train.shape[2]), activity_regularizer=l2(0.0001)))
model.add(Dropout(0.5))

# Add a 1D global max pooling layer to reduce the dimensions
model.add(GlobalMaxPooling1D())

# Add a dense layer with activation function
model.add(Dense(64, activation='relu'))

# Add the final dense layer for prediction
model.add(Dense(6, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(A_train, yt, epochs=15, batch_size=32, shuffle=True, validation_split=0.2)

Epoch 1/15
24/24 [==============================] - 5s 136ms/step - loss: 2.6912 - accuracy: 0.3460 - val_loss: 2.4468 - val_accuracy: 0.4293
Epoch 2/15
24/24 [==============================] - 3s 113ms/step - loss: 2.3399 - accuracy: 0.4076 - val_loss: 2.3293 - val_accuracy: 0.4450
Epoch 3/15
24/24 [==============================] - 3s 113ms/step - loss: 2.1956 - accuracy: 0.4797 - val_loss: 2.3324 - val_accuracy: 0.4136
Epoch 4/15
24/24 [==============================] - 3s 114ms/step - loss: 2.1585 - accuracy: 0.4823 - val_loss: 2.3043 - val_accuracy: 0.4346
Epoch 5/15
24/24 [==============================] - 3s 114ms/step - loss: 2.0798 - accuracy: 0.4980 - val_loss: 2.3422 - val_accuracy: 0.3089
Epoch 6/15
24/24 [==============================] - 3s 115ms/step - loss: 2.0664 - accuracy: 0.4980 - val_loss: 2.3462 - val_accuracy: 0.2932
Epoch 7/15
24/24 [==============================] - 3s 114ms/step - loss: 2.0091 - accuracy: 0.5518 - val_loss: 2.2999 - val_accuracy: 0.4293
Epoch 

In [ ]:
y_pred = model.predict(A_test)
y_pred = np.argmax(y_pred, axis=1)
y_arg = np.argmax(yt2, axis=1)

test_loss, test_acc = model.evaluate(A_test, yt2)

print('Test accuracy:', test_acc)

f1 = f1_score(y_arg, y_pred, average='micro')
print("F1-score:", f1)

print(confusion_matrix(y_arg, y_pred))

10/10 [==============================] - 0s 46ms/step - loss: 2.3160 - accuracy: 0.2327
Test accuracy: 0.23270440101623535
F1-score: 0.23270440251572327
[[  2  70   1   0   0   0]
 [  0  47   0   5   0   0]
 [  0  38   0   9   0   0]
 [  2 104   1  25   0   0]
 [  0   1   0   0   0   0]
 [  0  11   0   2   0   0]]


In [ ]:
## thats really low, lets try stronger regularization

model = Sequential()

# First LSTM layer with dropout
model.add(LSTM(512, return_sequences=True, input_shape=(A_train.shape[1], A_train.shape[2]), activity_regularizer=l2(0.01)))
model.add(Dropout(0.5))

# Add a 1D global max pooling layer to reduce the dimensions
model.add(GlobalMaxPooling1D())

# Add a dense layer with activation function
model.add(Dense(64, activation='relu'))

# Add the final dense layer for prediction
model.add(Dense(6, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(A_train, yt, epochs=15, batch_size=32, shuffle=True, validation_split=0.2)

Epoch 1/15
24/24 [==============================] - 5s 137ms/step - loss: 55.3234 - accuracy: 0.3237 - val_loss: 35.4376 - val_accuracy: 0.4136
Epoch 2/15
24/24 [==============================] - 3s 114ms/step - loss: 28.1197 - accuracy: 0.3893 - val_loss: 26.5443 - val_accuracy: 0.4188
Epoch 3/15
24/24 [==============================] - 3s 114ms/step - loss: 22.4982 - accuracy: 0.4364 - val_loss: 22.9081 - val_accuracy: 0.3246
Epoch 4/15
24/24 [==============================] - 3s 114ms/step - loss: 19.1667 - accuracy: 0.5072 - val_loss: 20.2304 - val_accuracy: 0.4031
Epoch 5/15
24/24 [==============================] - 3s 114ms/step - loss: 17.1669 - accuracy: 0.5256 - val_loss: 18.6921 - val_accuracy: 0.3717
Epoch 6/15
24/24 [==============================] - 3s 115ms/step - loss: 15.5878 - accuracy: 0.5963 - val_loss: 17.3589 - val_accuracy: 0.3351
Epoch 7/15
24/24 [==============================] - 3s 115ms/step - loss: 14.5962 - accuracy: 0.6448 - val_loss: 16.6963 - val_accuracy:

In [ ]:
y_pred = model.predict(A_test)
y_pred = np.argmax(y_pred, axis=1)
y_arg = np.argmax(yt2, axis=1)

test_loss, test_acc = model.evaluate(A_test, yt2)

print('Test accuracy:', test_acc)

f1 = f1_score(y_arg, y_pred, average='weighted')
print("F1-score:", f1)

print(confusion_matrix(y_arg, y_pred))

10/10 [==============================] - 1s 46ms/step - loss: 19.5917 - accuracy: 0.3805
Test accuracy: 0.3805031478404999
F1-score: 0.3703900415386944
[[27 24  5 17  0  0]
 [ 8 26  6 12  0  0]
 [10 10  5 21  0  1]
 [27 29 12 63  0  1]
 [ 0  1  0  0  0  0]
 [ 2  4  1  6  0  0]]


In [ ]:
## This strenghtened the results, lets try one more.

## thats really low, lets try stronger regularization

model = Sequential()

# First LSTM layer with dropout
model.add(LSTM(512, return_sequences=True, input_shape=(A_train.shape[1], A_train.shape[2]), activity_regularizer=l2(0.1)))
model.add(Dropout(0.5))

# Add a 1D global max pooling layer to reduce the dimensions
model.add(GlobalMaxPooling1D())

# Add a dense layer with activation function
model.add(Dense(64, activation='relu'))

# Add the final dense layer for prediction
model.add(Dense(6, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(A_train, yt, epochs=15, batch_size=32, shuffle=True, validation_split=0.2)

Epoch 1/15
24/24 [==============================] - 5s 140ms/step - loss: 503.3642 - accuracy: 0.3329 - val_loss: 308.7408 - val_accuracy: 0.2565
Epoch 2/15
24/24 [==============================] - 3s 117ms/step - loss: 260.5023 - accuracy: 0.3447 - val_loss: 224.6711 - val_accuracy: 0.4084
Epoch 3/15
24/24 [==============================] - 3s 117ms/step - loss: 165.7295 - accuracy: 0.3866 - val_loss: 166.4971 - val_accuracy: 0.2984
Epoch 4/15
24/24 [==============================] - 3s 118ms/step - loss: 138.7918 - accuracy: 0.3932 - val_loss: 147.1098 - val_accuracy: 0.3874
Epoch 5/15
24/24 [==============================] - 3s 119ms/step - loss: 116.7927 - accuracy: 0.4089 - val_loss: 128.2084 - val_accuracy: 0.3560
Epoch 6/15
24/24 [==============================] - 3s 119ms/step - loss: 100.2620 - accuracy: 0.4325 - val_loss: 115.3066 - val_accuracy: 0.4031
Epoch 7/15
24/24 [==============================] - 3s 120ms/step - loss: 89.8968 - accuracy: 0.4640 - val_loss: 104.7493 - 

In [ ]:
y_pred = model.predict(A_test)
y_pred = np.argmax(y_pred, axis=1)
y_arg = np.argmax(yt2, axis=1)

test_loss, test_acc = model.evaluate(A_test, yt2)

print('Test accuracy:', test_acc)

f1 = f1_score(y_arg, y_pred, average='weighted')
print("F1-score:", f1)

print(confusion_matrix(y_arg, y_pred))

10/10 [==============================] - 0s 46ms/step - loss: 66.4683 - accuracy: 0.3522
Test accuracy: 0.3522012531757355
F1-score: 0.32956799292763966
[[31 29  0 13  0  0]
 [19 16  0 17  0  0]
 [12 16  0 19  0  0]
 [41 26  0 65  0  0]
 [ 1  0  0  0  0  0]
 [ 8  0  0  5  0  0]]


In [ ]:
# Best parameters: L1(0.001), no regularization, L2(0.01)

# LSTM units

In [ ]:
#256 units

# Define the LSTM model
model = Sequential()

# First LSTM layer with dropout
model.add(LSTM(256, return_sequences=True, input_shape=(A_train.shape[1], A_train.shape[2])))
model.add(Dropout(0.5))

# Reshape the output to 3D
#model.add(Reshape((1, 128)))

# Add a 1D global max pooling layer to reduce the dimensions
model.add(GlobalMaxPooling1D())

# Add a dense layer with activation function
model.add(Dense(64, activation='relu'))

# Add the final dense layer for prediction
model.add(Dense(6, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(A_train, yt, epochs=15, batch_size=32, shuffle=True, validation_split=0.2)

Epoch 1/15
24/24 [==============================] - 3s 83ms/step - loss: 1.6747 - accuracy: 0.3473 - val_loss: 1.5465 - val_accuracy: 0.2775
Epoch 2/15
24/24 [==============================] - 1s 58ms/step - loss: 1.4352 - accuracy: 0.3971 - val_loss: 1.4956 - val_accuracy: 0.4136
Epoch 3/15
24/24 [==============================] - 1s 57ms/step - loss: 1.3721 - accuracy: 0.4430 - val_loss: 1.4741 - val_accuracy: 0.4136
Epoch 4/15
24/24 [==============================] - 1s 57ms/step - loss: 1.3122 - accuracy: 0.4784 - val_loss: 1.5413 - val_accuracy: 0.2618
Epoch 5/15
24/24 [==============================] - 1s 59ms/step - loss: 1.3113 - accuracy: 0.4666 - val_loss: 1.4719 - val_accuracy: 0.3979
Epoch 6/15
24/24 [==============================] - 1s 57ms/step - loss: 1.2682 - accuracy: 0.4889 - val_loss: 1.5145 - val_accuracy: 0.3770
Epoch 7/15
24/24 [==============================] - 1s 59ms/step - loss: 1.2431 - accuracy: 0.5046 - val_loss: 1.4917 - val_accuracy: 0.4241
Epoch 8/15
24

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score, f1_score


y_pred = model.predict(A_test)
y_pred = np.argmax(y_pred, axis=1)
y_arg = np.argmax(yt2, axis=1)

test_loss, test_acc = model.evaluate(A_test, yt2)

print('Test accuracy:', test_acc)

f1 = f1_score(y_arg, y_pred, average='weighted')
print("F1-score:", f1)

print(confusion_matrix(y_arg, y_pred))

10/10 [==============================] - 0s 31ms/step - loss: 1.5238 - accuracy: 0.3082
Test accuracy: 0.30817610025405884
F1-score: 0.2668499502540016
[[60 12  0  1  0  0]
 [39  9  2  2  0  0]
 [25  9  7  6  0  0]
 [80 22  8 22  0  0]
 [ 1  0  0  0  0  0]
 [ 8  3  0  2  0  0]]


In [ ]:
#128 units

# Define the LSTM model
model = Sequential()

# First LSTM layer with dropout
model.add(LSTM(128, return_sequences=True, input_shape=(A_train.shape[1], A_train.shape[2])))
model.add(Dropout(0.5))

# Reshape the output to 3D
#model.add(Reshape((1, 128)))

# Add a 1D global max pooling layer to reduce the dimensions
model.add(GlobalMaxPooling1D())

# Add a dense layer with activation function
model.add(Dense(64, activation='relu'))

# Add the final dense layer for prediction
model.add(Dense(6, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(A_train, yt, epochs=15, batch_size=32, shuffle=True, validation_split=0.2)

Epoch 1/15
24/24 [==============================] - 3s 75ms/step - loss: 1.7172 - accuracy: 0.2857 - val_loss: 1.4437 - val_accuracy: 0.4136
Epoch 2/15
24/24 [==============================] - 1s 51ms/step - loss: 1.4154 - accuracy: 0.4155 - val_loss: 1.4761 - val_accuracy: 0.4136
Epoch 3/15
24/24 [==============================] - 1s 50ms/step - loss: 1.3775 - accuracy: 0.4194 - val_loss: 1.4714 - val_accuracy: 0.4136
Epoch 4/15
24/24 [==============================] - 1s 54ms/step - loss: 1.3522 - accuracy: 0.4312 - val_loss: 1.4811 - val_accuracy: 0.4136
Epoch 5/15
24/24 [==============================] - 1s 54ms/step - loss: 1.3388 - accuracy: 0.4574 - val_loss: 1.4812 - val_accuracy: 0.4136
Epoch 6/15
24/24 [==============================] - 1s 52ms/step - loss: 1.3664 - accuracy: 0.4273 - val_loss: 1.4846 - val_accuracy: 0.3560
Epoch 7/15
24/24 [==============================] - 1s 50ms/step - loss: 1.3270 - accuracy: 0.4522 - val_loss: 1.4685 - val_accuracy: 0.4136
Epoch 8/15
24

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score, f1_score


y_pred = model.predict(A_test)
y_pred = np.argmax(y_pred, axis=1)
y_arg = np.argmax(yt2, axis=1)

test_loss, test_acc = model.evaluate(A_test, yt2)

print('Test accuracy:', test_acc)

f1 = f1_score(y_arg, y_pred, average='weighted')
print("F1-score:", f1)

print(confusion_matrix(y_arg, y_pred))

10/10 [==============================] - 0s 28ms/step - loss: 1.4739 - accuracy: 0.4497
Test accuracy: 0.44968554377555847
F1-score: 0.34997117577762743
[[ 12   2   3  56   0   0]
 [  1   6   1  44   0   0]
 [  6   1   2  38   0   0]
 [  4   4   1 123   0   0]
 [  1   0   0   0   0   0]
 [  2   0   1  10   0   0]]


In [ ]:
#64 units

# Define the LSTM model
model = Sequential()

# First LSTM layer with dropout
model.add(LSTM(64, return_sequences=True, input_shape=(A_train.shape[1], A_train.shape[2])))
model.add(Dropout(0.5))

# Reshape the output to 3D
#model.add(Reshape((1, 128)))

# Add a 1D global max pooling layer to reduce the dimensions
model.add(GlobalMaxPooling1D())

# Add a dense layer with activation function
model.add(Dense(64, activation='relu'))

# Add the final dense layer for prediction
model.add(Dense(6, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(A_train, yt, epochs=15, batch_size=32, shuffle=True, validation_split=0.2)

Epoch 1/15
24/24 [==============================] - 3s 76ms/step - loss: 1.6823 - accuracy: 0.3316 - val_loss: 1.4596 - val_accuracy: 0.4136
Epoch 2/15
24/24 [==============================] - 1s 51ms/step - loss: 1.4222 - accuracy: 0.4155 - val_loss: 1.5051 - val_accuracy: 0.4503
Epoch 3/15
24/24 [==============================] - 1s 50ms/step - loss: 1.3971 - accuracy: 0.4325 - val_loss: 1.4979 - val_accuracy: 0.4136
Epoch 4/15
24/24 [==============================] - 1s 51ms/step - loss: 1.4147 - accuracy: 0.4194 - val_loss: 1.4901 - val_accuracy: 0.3874
Epoch 5/15
24/24 [==============================] - 1s 43ms/step - loss: 1.3812 - accuracy: 0.4207 - val_loss: 1.4992 - val_accuracy: 0.3874
Epoch 6/15
24/24 [==============================] - 1s 44ms/step - loss: 1.3649 - accuracy: 0.4404 - val_loss: 1.4928 - val_accuracy: 0.3979
Epoch 7/15
24/24 [==============================] - 1s 43ms/step - loss: 1.3605 - accuracy: 0.4456 - val_loss: 1.4765 - val_accuracy: 0.4136
Epoch 8/15
24

In [ ]:


y_pred = model.predict(A_test)
y_pred = np.argmax(y_pred, axis=1)
y_arg = np.argmax(yt2, axis=1)

test_loss, test_acc = model.evaluate(A_test, yt2)

print('Test accuracy:', test_acc)

f1 = f1_score(y_arg, y_pred, average='weighted')
print("F1-score:", f1)

print(confusion_matrix(y_arg, y_pred))

10/10 [==============================] - 0s 25ms/step - loss: 1.4780 - accuracy: 0.4025
Test accuracy: 0.402515709400177
F1-score: 0.23879584481662075
[[  0   1   0  72   0   0]
 [  0   0   0  52   0   0]
 [  0   0   0  47   0   0]
 [  1   1   2 128   0   0]
 [  0   0   0   1   0   0]
 [  0   0   0  13   0   0]]


In [ ]:
# 128/512 look best.

# Dropout

In [ ]:
# 0.2

model = Sequential()

# First LSTM layer with dropout
model.add(LSTM(512, return_sequences=True, input_shape=(A_train.shape[1], A_train.shape[2])))
model.add(Dropout(0.2))

# Reshape the output to 3D
#model.add(Reshape((1, 128)))

# Add a 1D global max pooling layer to reduce the dimensions
model.add(GlobalMaxPooling1D())

# Add a dense layer with activation function
model.add(Dense(64, activation='relu'))

# Add the final dense layer for prediction
model.add(Dense(6, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(A_train, yt, epochs=15, batch_size=32, shuffle=True, validation_split=0.2)

Epoch 1/15
24/24 [==============================] - 5s 134ms/step - loss: 1.5386 - accuracy: 0.3617 - val_loss: 1.5009 - val_accuracy: 0.3089
Epoch 2/15
24/24 [==============================] - 3s 110ms/step - loss: 1.4070 - accuracy: 0.4181 - val_loss: 1.4574 - val_accuracy: 0.4136
Epoch 3/15
24/24 [==============================] - 3s 111ms/step - loss: 1.3507 - accuracy: 0.4142 - val_loss: 1.4433 - val_accuracy: 0.4136
Epoch 4/15
24/24 [==============================] - 3s 111ms/step - loss: 1.3168 - accuracy: 0.4430 - val_loss: 1.4721 - val_accuracy: 0.4136
Epoch 5/15
24/24 [==============================] - 3s 112ms/step - loss: 1.3091 - accuracy: 0.4404 - val_loss: 1.5649 - val_accuracy: 0.2408
Epoch 6/15
24/24 [==============================] - 3s 112ms/step - loss: 1.3352 - accuracy: 0.4587 - val_loss: 1.4349 - val_accuracy: 0.4188
Epoch 7/15
24/24 [==============================] - 3s 113ms/step - loss: 1.2284 - accuracy: 0.4653 - val_loss: 1.4244 - val_accuracy: 0.4136
Epoch 

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score, f1_score

y_pred = model.predict(A_test)
y_pred = np.argmax(y_pred, axis=1)
y_arg = np.argmax(yt2, axis=1)

test_loss, test_acc = model.evaluate(A_test, yt2)

print('Test accuracy:', test_acc)

f1 = f1_score(y_arg, y_pred, average='weighted')
print("F1-score:", f1)

print(confusion_matrix(y_arg, y_pred))

10/10 [==============================] - 0s 46ms/step - loss: 1.4954 - accuracy: 0.3302
Test accuracy: 0.33018869161605835
F1-score: 0.29410823592632335
[[ 0 61  0 12  0  0]
 [ 0 41  0 11  0  0]
 [ 0 25  9 13  0  0]
 [ 0 72  5 55  0  0]
 [ 0  1  0  0  0  0]
 [ 0  6  0  7  0  0]]


In [ ]:
# 0.2

model = Sequential()

# First LSTM layer with dropout
model.add(LSTM(512, return_sequences=True, input_shape=(A_train.shape[1], A_train.shape[2])))
model.add(Dropout(0.2))

# Reshape the output to 3D
#model.add(Reshape((1, 128)))

# Add a 1D global max pooling layer to reduce the dimensions
model.add(GlobalMaxPooling1D())

# Add a dense layer with activation function
model.add(Dense(64, activation='relu'))

# Add the final dense layer for prediction
model.add(Dense(6, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(A_train, yt, epochs=15, batch_size=32, shuffle=True, validation_split=0.2)

Epoch 1/15
24/24 [==============================] - 5s 142ms/step - loss: 1.6056 - accuracy: 0.3657 - val_loss: 1.4088 - val_accuracy: 0.4136
Epoch 2/15
24/24 [==============================] - 3s 114ms/step - loss: 1.4117 - accuracy: 0.4181 - val_loss: 1.4668 - val_accuracy: 0.3874
Epoch 3/15
24/24 [==============================] - 3s 115ms/step - loss: 1.3605 - accuracy: 0.4338 - val_loss: 1.5023 - val_accuracy: 0.4293
Epoch 4/15
24/24 [==============================] - 3s 115ms/step - loss: 1.2943 - accuracy: 0.4718 - val_loss: 1.4367 - val_accuracy: 0.4031
Epoch 5/15
24/24 [==============================] - 3s 116ms/step - loss: 1.2372 - accuracy: 0.5216 - val_loss: 1.4513 - val_accuracy: 0.4188
Epoch 6/15
24/24 [==============================] - 3s 118ms/step - loss: 1.2110 - accuracy: 0.5177 - val_loss: 1.3890 - val_accuracy: 0.4555
Epoch 7/15
24/24 [==============================] - 3s 116ms/step - loss: 1.1608 - accuracy: 0.5636 - val_loss: 1.3916 - val_accuracy: 0.4346
Epoch 

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score, f1_score

y_pred = model.predict(A_test)
y_pred = np.argmax(y_pred, axis=1)
y_arg = np.argmax(yt2, axis=1)

test_loss, test_acc = model.evaluate(A_test, yt2)

print('Test accuracy:', test_acc)

f1 = f1_score(y_arg, y_pred, average='weighted')
print("F1-score:", f1)

print(confusion_matrix(y_arg, y_pred))

10/10 [==============================] - 0s 45ms/step - loss: 1.3536 - accuracy: 0.4623
Test accuracy: 0.46226415038108826
F1-score: 0.36342052410356895
[[ 19   0   3  51   0   0]
 [ 16   0   4  32   0   0]
 [  3   0   6  38   0   0]
 [  3   0   7 122   0   0]
 [  1   0   0   0   0   0]
 [  2   0   0  11   0   0]]


In [ ]:
# 0.7
model = Sequential()
model.add(LSTM(512, return_sequences=True, input_shape=(A_train.shape[1], A_train.shape[2])))
model.add(Dropout(0.7))

# Reshape the output to 3D
#model.add(Reshape((1, 128)))

# Add a 1D global max pooling layer to reduce the dimensions
model.add(GlobalMaxPooling1D())

# Add a dense layer with activation function
model.add(Dense(64, activation='relu'))

# Add the final dense layer for prediction
model.add(Dense(6, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(A_train, yt, epochs=15, batch_size=32, shuffle=True, validation_split=0.2)

Epoch 1/15
24/24 [==============================] - 5s 133ms/step - loss: 1.9276 - accuracy: 0.2896 - val_loss: 1.5697 - val_accuracy: 0.3717
Epoch 2/15
24/24 [==============================] - 3s 111ms/step - loss: 1.3882 - accuracy: 0.4233 - val_loss: 1.5186 - val_accuracy: 0.4188
Epoch 3/15
24/24 [==============================] - 3s 113ms/step - loss: 1.3673 - accuracy: 0.4286 - val_loss: 1.5209 - val_accuracy: 0.3665
Epoch 4/15
24/24 [==============================] - 3s 111ms/step - loss: 1.2779 - accuracy: 0.4548 - val_loss: 1.5262 - val_accuracy: 0.4241
Epoch 5/15
24/24 [==============================] - 3s 112ms/step - loss: 1.2325 - accuracy: 0.5007 - val_loss: 1.5717 - val_accuracy: 0.3194
Epoch 6/15
24/24 [==============================] - 3s 112ms/step - loss: 1.1505 - accuracy: 0.5491 - val_loss: 1.5336 - val_accuracy: 0.3560
Epoch 7/15
24/24 [==============================] - 3s 114ms/step - loss: 1.1821 - accuracy: 0.5125 - val_loss: 1.5394 - val_accuracy: 0.3770
Epoch 

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score, f1_score

y_pred = model.predict(A_test)
y_pred = np.argmax(y_pred, axis=1)
y_arg = np.argmax(yt2, axis=1)

test_loss, test_acc = model.evaluate(A_test, yt2)

print('Test accuracy:', test_acc)

f1 = f1_score(y_arg, y_pred, average='weighted')
print("F1-score:", f1)

print(confusion_matrix(y_arg, y_pred))

10/10 [==============================] - 0s 45ms/step - loss: 1.5285 - accuracy: 0.4245
Test accuracy: 0.4245283007621765
F1-score: 0.30161696514230896
[[  0  26   0  47   0   0]
 [  0  16   0  36   0   0]
 [  0  10   0  37   0   0]
 [  0  13   0 119   0   0]
 [  0   0   0   1   0   0]
 [  0   1   0  12   0   0]]


# Dense layers/units

In [ ]:
# 1 Layer

# Define the LSTM model
model = Sequential()

# First LSTM layer with dropout
model.add(LSTM(512, return_sequences=True, input_shape=(A_train.shape[1], A_train.shape[2])))
model.add(Dropout(0.5))

# Reshape the output to 3D
#model.add(Reshape((1, 128)))

# Add a 1D global max pooling layer to reduce the dimensions
model.add(GlobalMaxPooling1D())

# Add the final dense layer for prediction
model.add(Dense(6, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(A_train, yt, epochs=15, batch_size=32, shuffle=True, validation_split=0.2)

Epoch 1/15
24/24 [==============================] - 5s 135ms/step - loss: 1.6161 - accuracy: 0.3670 - val_loss: 1.4850 - val_accuracy: 0.3246
Epoch 2/15
24/24 [==============================] - 3s 113ms/step - loss: 1.2991 - accuracy: 0.4797 - val_loss: 1.4502 - val_accuracy: 0.4188
Epoch 3/15
24/24 [==============================] - 3s 113ms/step - loss: 1.2358 - accuracy: 0.5033 - val_loss: 1.4656 - val_accuracy: 0.3822
Epoch 4/15
24/24 [==============================] - 3s 113ms/step - loss: 1.1581 - accuracy: 0.5609 - val_loss: 1.4731 - val_accuracy: 0.3508
Epoch 5/15
24/24 [==============================] - 3s 113ms/step - loss: 1.1138 - accuracy: 0.6147 - val_loss: 1.5106 - val_accuracy: 0.3403
Epoch 6/15
24/24 [==============================] - 3s 114ms/step - loss: 1.0604 - accuracy: 0.6173 - val_loss: 1.5744 - val_accuracy: 0.1728
Epoch 7/15
24/24 [==============================] - 3s 116ms/step - loss: 1.0777 - accuracy: 0.6055 - val_loss: 1.5549 - val_accuracy: 0.3037
Epoch 

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score, f1_score

y_pred = model.predict(A_test)
y_pred = np.argmax(y_pred, axis=1)
y_arg = np.argmax(yt2, axis=1)

test_loss, test_acc = model.evaluate(A_test, yt2)

print('Test accuracy:', test_acc)

f1 = f1_score(y_arg, y_pred, average='weighted')
print("F1-score:", f1)

print(confusion_matrix(y_arg, y_pred))

10/10 [==============================] - 0s 45ms/step - loss: 1.4552 - accuracy: 0.4528
Test accuracy: 0.4528301954269409
F1-score: 0.38277815036232216
[[40  2  1 30  0  0]
 [24  0  2 26  0  0]
 [10  1  6 30  0  0]
 [27  2  5 98  0  0]
 [ 1  0  0  0  0  0]
 [ 4  1  0  8  0  0]]


In [ ]:
# 3 Layers

In [ ]:
# Define the LSTM model
model = Sequential()

# First LSTM layer with dropout
model.add(LSTM(512, return_sequences=True, input_shape=(A_train.shape[1], A_train.shape[2])))
model.add(Dropout(0.5))

# Reshape the output to 3D
#model.add(Reshape((1, 128)))

# Add a 1D global max pooling layer to reduce the dimensions
model.add(GlobalMaxPooling1D())

# Add a dense layer with activation function
model.add(Dense(128, activation='relu'))
# Add a dense layer with activation function
model.add(Dense(64, activation='relu'))

# Add the final dense layer for prediction
model.add(Dense(6, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(A_train, yt, epochs=15, batch_size=32, shuffle=True, validation_split=0.2)

Epoch 1/15
24/24 [==============================] - 5s 134ms/step - loss: 1.8150 - accuracy: 0.3277 - val_loss: 1.4958 - val_accuracy: 0.2461
Epoch 2/15
24/24 [==============================] - 3s 111ms/step - loss: 1.4322 - accuracy: 0.4194 - val_loss: 1.5026 - val_accuracy: 0.4136
Epoch 3/15
24/24 [==============================] - 3s 112ms/step - loss: 1.3612 - accuracy: 0.4233 - val_loss: 1.5016 - val_accuracy: 0.4136
Epoch 4/15
24/24 [==============================] - 3s 112ms/step - loss: 1.3257 - accuracy: 0.4509 - val_loss: 1.4645 - val_accuracy: 0.4136
Epoch 5/15
24/24 [==============================] - 3s 114ms/step - loss: 1.3107 - accuracy: 0.4220 - val_loss: 1.5422 - val_accuracy: 0.1990
Epoch 6/15
24/24 [==============================] - 3s 114ms/step - loss: 1.3104 - accuracy: 0.4574 - val_loss: 1.4879 - val_accuracy: 0.3403
Epoch 7/15
24/24 [==============================] - 3s 116ms/step - loss: 1.2480 - accuracy: 0.4705 - val_loss: 1.4542 - val_accuracy: 0.4346
Epoch 

In [ ]:

y_pred = model.predict(A_test)
y_pred = np.argmax(y_pred, axis=1)
y_arg = np.argmax(yt2, axis=1)

test_loss, test_acc = model.evaluate(A_test, yt2)

print('Test accuracy:', test_acc)

f1 = f1_score(y_arg, y_pred, average='weighted')
print("F1-score:", f1)

print(confusion_matrix(y_arg, y_pred))

10/10 [==============================] - 0s 46ms/step - loss: 1.4366 - accuracy: 0.4528
Test accuracy: 0.4528301954269409
F1-score: 0.3574870346650642
[[ 10  12   1  50   0   0]
 [  3  10   0  39   0   0]
 [  0   8   2  37   0   0]
 [  4   5   1 122   0   0]
 [  0   0   0   1   0   0]
 [  0   1   0  12   0   0]]


In [ ]:
### All similarly good hmm....

In [ ]:
# Different dense units for 2 layers:

In [ ]:
# Define the LSTM model
model = Sequential()

# First LSTM layer with dropout
model.add(LSTM(512, return_sequences=True, input_shape=(A_train.shape[1], A_train.shape[2])))
model.add(Dropout(0.5))

# Reshape the output to 3D
#model.add(Reshape((1, 128)))

# Add a 1D global max pooling layer to reduce the dimensions
model.add(GlobalMaxPooling1D())

# Add a dense layer with activation function
model.add(Dense(128, activation='relu'))

# Add the final dense layer for prediction
model.add(Dense(6, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(A_train, yt, epochs=15, batch_size=32, shuffle=True, validation_split=0.2)

Epoch 1/15
24/24 [==============================] - 5s 137ms/step - loss: 1.8664 - accuracy: 0.3526 - val_loss: 1.4930 - val_accuracy: 0.3141
Epoch 2/15
24/24 [==============================] - 3s 110ms/step - loss: 1.4351 - accuracy: 0.4115 - val_loss: 1.5122 - val_accuracy: 0.3351
Epoch 3/15
24/24 [==============================] - 3s 111ms/step - loss: 1.3096 - accuracy: 0.4705 - val_loss: 1.5765 - val_accuracy: 0.1885
Epoch 4/15
24/24 [==============================] - 3s 112ms/step - loss: 1.3133 - accuracy: 0.4495 - val_loss: 1.4593 - val_accuracy: 0.4712
Epoch 5/15
24/24 [==============================] - 3s 112ms/step - loss: 1.2597 - accuracy: 0.4587 - val_loss: 1.4749 - val_accuracy: 0.3874
Epoch 6/15
24/24 [==============================] - 3s 114ms/step - loss: 1.2529 - accuracy: 0.4889 - val_loss: 1.4801 - val_accuracy: 0.3089
Epoch 7/15
24/24 [==============================] - 3s 114ms/step - loss: 1.1804 - accuracy: 0.5426 - val_loss: 1.5111 - val_accuracy: 0.4136
Epoch 

In [ ]:
y_pred = model.predict(A_test)
y_pred = np.argmax(y_pred, axis=1)
y_arg = np.argmax(yt2, axis=1)

test_loss, test_acc = model.evaluate(A_test, yt2)

print('Test accuracy:', test_acc)

f1 = f1_score(y_arg, y_pred, average='weighted')
print("F1-score:", f1)

print(confusion_matrix(y_arg, y_pred))

10/10 [==============================] - 0s 46ms/step - loss: 1.4121 - accuracy: 0.4245
Test accuracy: 0.4245283007621765
F1-score: 0.30196700844296703
[[  0  19   0  54   0   0]
 [  0  12   0  40   0   0]
 [  0   3   2  42   0   0]
 [  0  10   1 121   0   0]
 [  0   0   0   1   0   0]
 [  0   0   0  13   0   0]]


In [ ]:
# 256

# Define the LSTM model
model = Sequential()

# First LSTM layer with dropout
model.add(LSTM(512, return_sequences=True, input_shape=(A_train.shape[1], A_train.shape[2])))
model.add(Dropout(0.5))

# Reshape the output to 3D
#model.add(Reshape((1, 128)))

# Add a 1D global max pooling layer to reduce the dimensions
model.add(GlobalMaxPooling1D())

# Add a dense layer with activation function
model.add(Dense(256, activation='relu'))

# Add the final dense layer for prediction
model.add(Dense(6, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(A_train, yt, epochs=15, batch_size=32, shuffle=True, validation_split=0.2)

Epoch 1/15
24/24 [==============================] - 5s 137ms/step - loss: 2.2915 - accuracy: 0.3119 - val_loss: 1.5703 - val_accuracy: 0.4136
Epoch 2/15
24/24 [==============================] - 3s 114ms/step - loss: 1.5098 - accuracy: 0.4024 - val_loss: 1.5096 - val_accuracy: 0.4136
Epoch 3/15
24/24 [==============================] - 3s 114ms/step - loss: 1.4738 - accuracy: 0.3932 - val_loss: 1.4918 - val_accuracy: 0.4136
Epoch 4/15
24/24 [==============================] - 3s 114ms/step - loss: 1.3166 - accuracy: 0.4823 - val_loss: 1.5122 - val_accuracy: 0.3822
Epoch 5/15
24/24 [==============================] - 3s 114ms/step - loss: 1.2525 - accuracy: 0.5046 - val_loss: 1.4615 - val_accuracy: 0.4241
Epoch 6/15
24/24 [==============================] - 3s 116ms/step - loss: 1.2758 - accuracy: 0.4718 - val_loss: 1.5195 - val_accuracy: 0.2880
Epoch 7/15
24/24 [==============================] - 3s 115ms/step - loss: 1.2466 - accuracy: 0.4771 - val_loss: 1.5651 - val_accuracy: 0.2827
Epoch 

In [ ]:
y_pred = model.predict(A_test)
y_pred = np.argmax(y_pred, axis=1)
y_arg = np.argmax(yt2, axis=1)

test_loss, test_acc = model.evaluate(A_test, yt2)

print('Test accuracy:', test_acc)

f1 = f1_score(y_arg, y_pred, average='weighted')
print("F1-score:", f1)

print(confusion_matrix(y_arg, y_pred))

10/10 [==============================] - 0s 47ms/step - loss: 1.5029 - accuracy: 0.3899
Test accuracy: 0.3899371027946472
F1-score: 0.34846287127433245
[[61  2  5  5  0  0]
 [29  4  5 14  0  0]
 [11  2 18 16  0  0]
 [65  5 21 41  0  0]
 [ 1  0  0  0  0  0]
 [ 8  0  1  4  0  0]]


In [ ]:
# 32

# Define the LSTM model
model = Sequential()

# First LSTM layer with dropout
model.add(LSTM(512, return_sequences=True, input_shape=(A_train.shape[1], A_train.shape[2])))
model.add(Dropout(0.5))

# Reshape the output to 3D
#model.add(Reshape((1, 128)))

# Add a 1D global max pooling layer to reduce the dimensions
model.add(GlobalMaxPooling1D())

# Add a dense layer with activation function
model.add(Dense(32, activation='relu'))

# Add the final dense layer for prediction
model.add(Dense(6, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(A_train, yt, epochs=15, batch_size=32, shuffle=True, validation_split=0.2)

Epoch 1/15
24/24 [==============================] - 5s 134ms/step - loss: 1.5635 - accuracy: 0.3198 - val_loss: 1.4880 - val_accuracy: 0.4084
Epoch 2/15
24/24 [==============================] - 3s 113ms/step - loss: 1.4172 - accuracy: 0.4181 - val_loss: 1.4599 - val_accuracy: 0.4136
Epoch 3/15
24/24 [==============================] - 3s 113ms/step - loss: 1.3537 - accuracy: 0.4351 - val_loss: 1.5239 - val_accuracy: 0.2461
Epoch 4/15
24/24 [==============================] - 3s 113ms/step - loss: 1.2962 - accuracy: 0.4364 - val_loss: 1.4801 - val_accuracy: 0.4293
Epoch 5/15
24/24 [==============================] - 3s 113ms/step - loss: 1.2686 - accuracy: 0.4875 - val_loss: 1.4693 - val_accuracy: 0.4136
Epoch 6/15
24/24 [==============================] - 3s 114ms/step - loss: 1.2371 - accuracy: 0.4797 - val_loss: 1.5299 - val_accuracy: 0.2984
Epoch 7/15
24/24 [==============================] - 3s 116ms/step - loss: 1.2208 - accuracy: 0.5085 - val_loss: 1.4666 - val_accuracy: 0.4084
Epoch 

In [ ]:

y_pred = model.predict(A_test)
y_pred = np.argmax(y_pred, axis=1)
y_arg = np.argmax(yt2, axis=1)

test_loss, test_acc = model.evaluate(A_test, yt2)

print('Test accuracy:', test_acc)

f1 = f1_score(y_arg, y_pred, average='weighted')
print("F1-score:", f1)

print(confusion_matrix(y_arg, y_pred))

10/10 [==============================] - 0s 45ms/step - loss: 1.4606 - accuracy: 0.4025
Test accuracy: 0.402515709400177
F1-score: 0.37519844206026237
[[26 30  0 17  0  0]
 [13 20  0 19  0  0]
 [ 5 19  1 22  0  0]
 [20 31  0 81  0  0]
 [ 1  0  0  0  0  0]
 [ 8  1  0  4  0  0]]


# Pooling

In [ ]:
# With no pooling (just flatten instead):

In [ ]:
# Define the LSTM model
model = Sequential()

# First LSTM layer with dropout
model.add(LSTM(512, return_sequences=True, input_shape=(A_train.shape[1], A_train.shape[2])))
model.add(Dropout(0.5))

# Reshape the output to 3D
#model.add(Reshape((1, 128)))

# Flatten
model.add(Flatten())

# Add a dense layer with activation function
model.add(Dense(64, activation='relu'))

# Add the final dense layer for prediction
model.add(Dense(6, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(A_train, yt, epochs=15, batch_size=32, shuffle=True, validation_split=0.2)

Epoch 1/15
24/24 [==============================] - 6s 143ms/step - loss: 1.5461 - accuracy: 0.3670 - val_loss: 1.4820 - val_accuracy: 0.3927
Epoch 2/15
24/24 [==============================] - 3s 117ms/step - loss: 0.3436 - accuracy: 0.8834 - val_loss: 1.5936 - val_accuracy: 0.3560
Epoch 3/15
24/24 [==============================] - 3s 116ms/step - loss: 0.0673 - accuracy: 0.9869 - val_loss: 1.8564 - val_accuracy: 0.3822
Epoch 4/15
24/24 [==============================] - 3s 114ms/step - loss: 0.0166 - accuracy: 0.9987 - val_loss: 1.9438 - val_accuracy: 0.3927
Epoch 5/15
24/24 [==============================] - 3s 115ms/step - loss: 0.0037 - accuracy: 1.0000 - val_loss: 2.1404 - val_accuracy: 0.4136
Epoch 6/15
24/24 [==============================] - 3s 115ms/step - loss: 0.0013 - accuracy: 1.0000 - val_loss: 2.3061 - val_accuracy: 0.3979
Epoch 7/15
24/24 [==============================] - 3s 116ms/step - loss: 8.5261e-04 - accuracy: 1.0000 - val_loss: 2.2236 - val_accuracy: 0.4084
Ep

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score, f1_score

y_pred = model.predict(A_test)
y_pred = np.argmax(y_pred, axis=1)
y_arg = np.argmax(yt2, axis=1)

test_loss, test_acc = model.evaluate(A_test, yt2)

print('Test accuracy:', test_acc)

f1 = f1_score(y_arg, y_pred, average='weighted')
print("F1-score:", f1)

print(confusion_matrix(y_arg, y_pred))

10/10 [==============================] - 0s 47ms/step - loss: 2.7724 - accuracy: 0.4245
Test accuracy: 0.4245283007621765
F1-score: 0.3534062446120276
[[ 10  11   7  45   0   0]
 [  2   7   3  40   0   0]
 [  3   3   7  34   0   0]
 [  6   6   9 111   0   0]
 [  0   1   0   0   0   0]
 [  2   3   2   6   0   0]]


# Activation function

In [ ]:
### SIGMOID

# Define the LSTM model
model = Sequential()

# First LSTM layer with dropout
model.add(LSTM(512, return_sequences=True, input_shape=(A_train.shape[1], A_train.shape[2])))
model.add(Dropout(0.5))

# Reshape the output to 3D
#model.add(Reshape((1, 128)))

# Add a 1D global max pooling layer to reduce the dimensions
model.add(GlobalMaxPooling1D())

# Add a dense layer with activation function
model.add(Dense(64, activation='relu'))

# Add the final dense layer for prediction
model.add(Dense(6, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(A_train, yt, epochs=15, batch_size=32, shuffle=True, validation_split=0.2)

Epoch 1/15
24/24 [==============================] - 5s 134ms/step - loss: 1.7675 - accuracy: 0.3080 - val_loss: 1.5041 - val_accuracy: 0.3927
Epoch 2/15
24/24 [==============================] - 3s 111ms/step - loss: 1.4546 - accuracy: 0.3853 - val_loss: 1.4437 - val_accuracy: 0.4136
Epoch 3/15
24/24 [==============================] - 3s 112ms/step - loss: 1.3093 - accuracy: 0.4574 - val_loss: 1.4702 - val_accuracy: 0.4241
Epoch 4/15
24/24 [==============================] - 3s 111ms/step - loss: 1.2990 - accuracy: 0.4718 - val_loss: 1.5786 - val_accuracy: 0.2251
Epoch 5/15
24/24 [==============================] - 3s 112ms/step - loss: 1.2641 - accuracy: 0.5072 - val_loss: 1.4445 - val_accuracy: 0.4136
Epoch 6/15
24/24 [==============================] - 3s 113ms/step - loss: 1.1910 - accuracy: 0.5334 - val_loss: 1.5925 - val_accuracy: 0.1780
Epoch 7/15
24/24 [==============================] - 3s 114ms/step - loss: 1.1806 - accuracy: 0.5334 - val_loss: 1.4945 - val_accuracy: 0.4084
Epoch 

In [ ]:
y_pred = model.predict(A_test)
y_pred = np.argmax(y_pred, axis=1)
y_arg = np.argmax(yt2, axis=1)

test_loss, test_acc = model.evaluate(A_test, yt2)

print('Test accuracy:', test_acc)

f1 = f1_score(y_arg, y_pred, average='weighted')
print("F1-score:", f1)

print(confusion_matrix(y_arg, y_pred))

10/10 [==============================] - 0s 46ms/step - loss: 1.4878 - accuracy: 0.3208
Test accuracy: 0.3207547068595886
F1-score: 0.3134694761201023
[[11 44 11  7  0  0]
 [ 7 25 12  8  0  0]
 [ 2 10 27  8  0  0]
 [11 35 47 39  0  0]
 [ 1  0  0  0  0  0]
 [ 1  7  1  4  0  0]]


In [ ]:
### Tanh

# Define the LSTM model
model = Sequential()

# First LSTM layer with dropout
model.add(LSTM(512, return_sequences=True, input_shape=(A_train.shape[1], A_train.shape[2])))
model.add(Dropout(0.5))

# Reshape the output to 3D
#model.add(Reshape((1, 128)))

# Add a 1D global max pooling layer to reduce the dimensions
model.add(GlobalMaxPooling1D())

# Add a dense layer with activation function
model.add(Dense(64, activation='tanh'))

# Add the final dense layer for prediction
model.add(Dense(6, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(A_train, yt, epochs=15, batch_size=32, shuffle=True, validation_split=0.2)

Epoch 1/15
24/24 [==============================] - 5s 136ms/step - loss: 1.4974 - accuracy: 0.3919 - val_loss: 1.4219 - val_accuracy: 0.4136
Epoch 2/15
24/24 [==============================] - 3s 112ms/step - loss: 1.4064 - accuracy: 0.4010 - val_loss: 1.4280 - val_accuracy: 0.4136
Epoch 3/15
24/24 [==============================] - 3s 114ms/step - loss: 1.3491 - accuracy: 0.4273 - val_loss: 1.4031 - val_accuracy: 0.4136
Epoch 4/15
24/24 [==============================] - 3s 115ms/step - loss: 1.3414 - accuracy: 0.4325 - val_loss: 1.4629 - val_accuracy: 0.3298
Epoch 5/15
24/24 [==============================] - 3s 116ms/step - loss: 1.3056 - accuracy: 0.4941 - val_loss: 1.3944 - val_accuracy: 0.4136
Epoch 6/15
24/24 [==============================] - 3s 116ms/step - loss: 1.2868 - accuracy: 0.4784 - val_loss: 1.4724 - val_accuracy: 0.3927
Epoch 7/15
24/24 [==============================] - 3s 117ms/step - loss: 1.2614 - accuracy: 0.4889 - val_loss: 1.3882 - val_accuracy: 0.4136
Epoch 

In [ ]:
y_pred = model.predict(A_test)
y_pred = np.argmax(y_pred, axis=1)
y_arg = np.argmax(yt2, axis=1)

test_loss, test_acc = model.evaluate(A_test, yt2)

print('Test accuracy:', test_acc)

f1 = f1_score(y_arg, y_pred, average='weighted')
print("F1-score:", f1)

print(confusion_matrix(y_arg, y_pred))

10/10 [==============================] - 0s 47ms/step - loss: 1.3761 - accuracy: 0.4403
Test accuracy: 0.44025155901908875
F1-score: 0.316542580110638
[[ 12   1   0  60   0   0]
 [  7   2   0  43   0   0]
 [  3   0   0  44   0   0]
 [  5   1   0 126   0   0]
 [  1   0   0   0   0   0]
 [  1   0   0  12   0   0]]


In [ ]:
### LeakyRELU

# Define the LSTM model
model = Sequential()

# First LSTM layer with dropout
model.add(LSTM(512, return_sequences=True, input_shape=(A_train.shape[1], A_train.shape[2])))
model.add(Dropout(0.5))

# Reshape the output to 3D
#model.add(Reshape((1, 128)))

# Add a 1D global max pooling layer to reduce the dimensions
model.add(GlobalMaxPooling1D())

# Add a dense layer with activation function
model.add(Dense(64, activation='leaky_relu'))

# Add the final dense layer for prediction
model.add(Dense(6, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(A_train, yt, epochs=15, batch_size=32, shuffle=True, validation_split=0.2)

Epoch 1/15
24/24 [==============================] - 5s 135ms/step - loss: 1.7381 - accuracy: 0.3224 - val_loss: 1.5139 - val_accuracy: 0.3089
Epoch 2/15
24/24 [==============================] - 3s 113ms/step - loss: 1.3861 - accuracy: 0.4273 - val_loss: 1.4725 - val_accuracy: 0.4293
Epoch 3/15
24/24 [==============================] - 3s 112ms/step - loss: 1.2951 - accuracy: 0.4679 - val_loss: 1.5286 - val_accuracy: 0.2565
Epoch 4/15
24/24 [==============================] - 3s 114ms/step - loss: 1.2798 - accuracy: 0.4731 - val_loss: 1.4845 - val_accuracy: 0.3508
Epoch 5/15
24/24 [==============================] - 3s 112ms/step - loss: 1.2815 - accuracy: 0.4640 - val_loss: 1.4477 - val_accuracy: 0.4136
Epoch 6/15
24/24 [==============================] - 3s 114ms/step - loss: 1.2243 - accuracy: 0.5216 - val_loss: 1.4744 - val_accuracy: 0.3927
Epoch 7/15
24/24 [==============================] - 3s 114ms/step - loss: 1.1513 - accuracy: 0.5478 - val_loss: 1.4458 - val_accuracy: 0.4398
Epoch 

In [ ]:
y_pred = model.predict(A_test)
y_pred = np.argmax(y_pred, axis=1)
y_arg = np.argmax(yt2, axis=1)

test_loss, test_acc = model.evaluate(A_test, yt2)

print('Test accuracy:', test_acc)

f1 = f1_score(y_arg, y_pred, average='weighted')
print("F1-score:", f1)

print(confusion_matrix(y_arg, y_pred))

10/10 [==============================] - 0s 47ms/step - loss: 1.4164 - accuracy: 0.4308
Test accuracy: 0.4308176040649414
F1-score: 0.3094318212230697
[[  0  28   0  45   0   0]
 [  1  17   0  34   0   0]
 [  0  10   0  37   0   0]
 [  0  12   0 120   0   0]
 [  0   1   0   0   0   0]
 [  0   5   0   8   0   0]]


# Batch Size:

In [ ]:
# Define the LSTM model
model = Sequential()

# First LSTM layer with dropout
model.add(LSTM(512, return_sequences=True, input_shape=(A_train.shape[1], A_train.shape[2])))
model.add(Dropout(0.5))

# Reshape the output to 3D
#model.add(Reshape((1, 128)))

# Add a 1D global max pooling layer to reduce the dimensions
model.add(GlobalMaxPooling1D())

# Add a dense layer with activation function
model.add(Dense(64, activation='relu'))

# Add the final dense layer for prediction
model.add(Dense(6, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(A_train, yt, epochs=15, batch_size=64, shuffle=True, validation_split=0.2)

Epoch 1/15
12/12 [==============================] - 5s 258ms/step - loss: 1.6182 - accuracy: 0.3198 - val_loss: 1.4821 - val_accuracy: 0.4188
Epoch 2/15
12/12 [==============================] - 2s 209ms/step - loss: 1.3852 - accuracy: 0.4456 - val_loss: 1.5097 - val_accuracy: 0.3560
Epoch 3/15
12/12 [==============================] - 3s 210ms/step - loss: 1.2938 - accuracy: 0.4679 - val_loss: 1.4801 - val_accuracy: 0.4084
Epoch 4/15
12/12 [==============================] - 3s 213ms/step - loss: 1.2291 - accuracy: 0.4954 - val_loss: 1.5236 - val_accuracy: 0.3770
Epoch 5/15
12/12 [==============================] - 3s 212ms/step - loss: 1.1889 - accuracy: 0.5256 - val_loss: 1.4441 - val_accuracy: 0.4136
Epoch 6/15
12/12 [==============================] - 3s 212ms/step - loss: 1.1676 - accuracy: 0.5531 - val_loss: 1.5345 - val_accuracy: 0.3089
Epoch 7/15
12/12 [==============================] - 3s 215ms/step - loss: 1.1577 - accuracy: 0.5609 - val_loss: 1.5241 - val_accuracy: 0.3979
Epoch 

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score, f1_score

y_pred = model.predict(A_test)
y_pred = np.argmax(y_pred, axis=1)
y_arg = np.argmax(yt2, axis=1)

test_loss, test_acc = model.evaluate(A_test, yt2)

print('Test accuracy:', test_acc)

f1 = f1_score(y_arg, y_pred, average='weighted')
print("F1-score:", f1)

print(confusion_matrix(y_arg, y_pred))

10/10 [==============================] - 0s 44ms/step - loss: 1.5159 - accuracy: 0.2673
Test accuracy: 0.26729559898376465
F1-score: 0.24807853402706503
[[30 43  0  0  0  0]
 [20 29  2  1  0  0]
 [13 24  2  8  0  0]
 [46 57  5 24  0  0]
 [ 0  1  0  0  0  0]
 [ 6  5  0  2  0  0]]


In [ ]:
# Define the LSTM model
model = Sequential()

# First LSTM layer with dropout
model.add(LSTM(512, return_sequences=True, input_shape=(A_train.shape[1], A_train.shape[2])))
model.add(Dropout(0.5))

# Reshape the output to 3D
#model.add(Reshape((1, 128)))

# Add a 1D global max pooling layer to reduce the dimensions
model.add(GlobalMaxPooling1D())

# Add a dense layer with activation function
model.add(Dense(64, activation='relu'))

# Add the final dense layer for prediction
model.add(Dense(6, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(A_train, yt, epochs=15, batch_size=128, shuffle=True, validation_split=0.2)

Epoch 1/15
6/6 [==============================] - 4s 495ms/step - loss: 2.0587 - accuracy: 0.3434 - val_loss: 1.6851 - val_accuracy: 0.3822
Epoch 2/15
6/6 [==============================] - 2s 399ms/step - loss: 1.6261 - accuracy: 0.4246 - val_loss: 1.5319 - val_accuracy: 0.4136
Epoch 3/15
6/6 [==============================] - 2s 393ms/step - loss: 1.3441 - accuracy: 0.4574 - val_loss: 1.5037 - val_accuracy: 0.4084
Epoch 4/15
6/6 [==============================] - 2s 395ms/step - loss: 1.3063 - accuracy: 0.4810 - val_loss: 1.4869 - val_accuracy: 0.4660
Epoch 5/15
6/6 [==============================] - 2s 395ms/step - loss: 1.2349 - accuracy: 0.5426 - val_loss: 1.4638 - val_accuracy: 0.4136
Epoch 6/15
6/6 [==============================] - 2s 399ms/step - loss: 1.2091 - accuracy: 0.5334 - val_loss: 1.4945 - val_accuracy: 0.4346
Epoch 7/15
6/6 [==============================] - 2s 397ms/step - loss: 1.1628 - accuracy: 0.5636 - val_loss: 1.4954 - val_accuracy: 0.2880
Epoch 8/15
6/6 [====

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score, f1_score

y_pred = model.predict(A_test)
y_pred = np.argmax(y_pred, axis=1)
y_arg = np.argmax(yt2, axis=1)

test_loss, test_acc = model.evaluate(A_test, yt2)

print('Test accuracy:', test_acc)

f1 = f1_score(y_arg, y_pred, average='weighted')
print("F1-score:", f1)

print(confusion_matrix(y_arg, y_pred))

10/10 [==============================] - 0s 47ms/step - loss: 1.5141 - accuracy: 0.3050
Test accuracy: 0.3050314486026764
F1-score: 0.29024262742073187
[[14 55  0  4  0  0]
 [ 9 34  0  9  0  0]
 [ 5 25  0 17  0  0]
 [16 67  0 49  0  0]
 [ 0  1  0  0  0  0]
 [ 4  7  0  2  0  0]]


In [ ]:
# Define the LSTM model
model = Sequential()

# First LSTM layer with dropout
model.add(LSTM(512, return_sequences=True, input_shape=(A_train.shape[1], A_train.shape[2])))
model.add(Dropout(0.5))

# Reshape the output to 3D
#model.add(Reshape((1, 128)))

# Add a 1D global max pooling layer to reduce the dimensions
model.add(GlobalMaxPooling1D())

# Add a dense layer with activation function
model.add(Dense(64, activation='relu'))

# Add the final dense layer for prediction
model.add(Dense(6, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(A_train, yt, epochs=15, batch_size=16, shuffle=True, validation_split=0.2)

Epoch 1/15
48/48 [==============================] - 6s 87ms/step - loss: 1.6093 - accuracy: 0.3591 - val_loss: 1.4973 - val_accuracy: 0.4188
Epoch 2/15
48/48 [==============================] - 4s 78ms/step - loss: 1.4365 - accuracy: 0.3893 - val_loss: 1.4626 - val_accuracy: 0.4293
Epoch 3/15
48/48 [==============================] - 4s 78ms/step - loss: 1.3382 - accuracy: 0.4443 - val_loss: 1.4655 - val_accuracy: 0.3665
Epoch 4/15
48/48 [==============================] - 4s 79ms/step - loss: 1.3256 - accuracy: 0.4456 - val_loss: 1.5189 - val_accuracy: 0.1832
Epoch 5/15
48/48 [==============================] - 4s 78ms/step - loss: 1.2794 - accuracy: 0.4705 - val_loss: 1.4935 - val_accuracy: 0.4136
Epoch 6/15
48/48 [==============================] - 4s 79ms/step - loss: 1.2288 - accuracy: 0.5033 - val_loss: 1.5078 - val_accuracy: 0.3351
Epoch 7/15
48/48 [==============================] - 4s 79ms/step - loss: 1.2519 - accuracy: 0.5020 - val_loss: 1.4612 - val_accuracy: 0.4712
Epoch 8/15
48

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score, f1_score

y_pred = model.predict(A_test)
y_pred = np.argmax(y_pred, axis=1)
y_arg = np.argmax(yt2, axis=1)

test_loss, test_acc = model.evaluate(A_test, yt2)

print('Test accuracy:', test_acc)

f1 = f1_score(y_arg, y_pred, average='weighted')
print("F1-score:", f1)

print(confusion_matrix(y_arg, y_pred))

10/10 [==============================] - 0s 46ms/step - loss: 1.3934 - accuracy: 0.4560
Test accuracy: 0.45597484707832336
F1-score: 0.35427030505397944
[[ 38   0   0  35   0   0]
 [ 19   0   0  33   0   0]
 [ 11   0   0  36   0   0]
 [ 25   0   0 107   0   0]
 [  1   0   0   0   0   0]
 [  3   0   0  10   0   0]]


# Epochs

In [ ]:
# Define the LSTM model
model = Sequential()

# First LSTM layer with dropout
model.add(LSTM(512, return_sequences=True, input_shape=(A_train.shape[1], A_train.shape[2])))
model.add(Dropout(0.5))

# Reshape the output to 3D
#model.add(Reshape((1, 128)))

# Add a 1D global max pooling layer to reduce the dimensions
model.add(GlobalMaxPooling1D())

# Add a dense layer with activation function
model.add(Dense(64, activation='relu'))

# Add the final dense layer for prediction
model.add(Dense(6, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(A_train, yt, epochs=10, batch_size=32, shuffle=True, validation_split=0.2)

Epoch 1/10
24/24 [==============================] - 5s 137ms/step - loss: 1.7703 - accuracy: 0.3355 - val_loss: 1.6172 - val_accuracy: 0.1518
Epoch 2/10
24/24 [==============================] - 3s 116ms/step - loss: 1.4295 - accuracy: 0.3827 - val_loss: 1.5355 - val_accuracy: 0.2513
Epoch 3/10
24/24 [==============================] - 3s 113ms/step - loss: 1.3003 - accuracy: 0.4731 - val_loss: 1.5138 - val_accuracy: 0.3927
Epoch 4/10
24/24 [==============================] - 3s 115ms/step - loss: 1.2901 - accuracy: 0.4915 - val_loss: 1.5179 - val_accuracy: 0.4084
Epoch 5/10
24/24 [==============================] - 3s 116ms/step - loss: 1.2357 - accuracy: 0.4993 - val_loss: 1.4839 - val_accuracy: 0.4136
Epoch 6/10
24/24 [==============================] - 3s 114ms/step - loss: 1.2042 - accuracy: 0.5242 - val_loss: 1.5129 - val_accuracy: 0.3298
Epoch 7/10
24/24 [==============================] - 3s 117ms/step - loss: 1.1980 - accuracy: 0.5269 - val_loss: 1.4921 - val_accuracy: 0.3665
Epoch 

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score, f1_score

y_pred = model.predict(A_test)
y_pred = np.argmax(y_pred, axis=1)
y_arg = np.argmax(yt2, axis=1)

test_loss, test_acc = model.evaluate(A_test, yt2)

print('Test accuracy:', test_acc)

f1 = f1_score(y_arg, y_pred, average='weighted')
print("F1-score:", f1)

print(confusion_matrix(y_arg, y_pred))

10/10 [==============================] - 0s 47ms/step - loss: 1.4585 - accuracy: 0.4528
Test accuracy: 0.4528301954269409
F1-score: 0.39537545671707697
[[36 14  0 23  0  0]
 [18 12  0 22  0  0]
 [ 8  8  0 31  0  0]
 [30  6  0 96  0  0]
 [ 1  0  0  0  0  0]
 [ 3  3  0  7  0  0]]


In [ ]:
# Define the LSTM model
model = Sequential()

# First LSTM layer with dropout
model.add(LSTM(512, return_sequences=True, input_shape=(A_train.shape[1], A_train.shape[2])))
model.add(Dropout(0.5))

# Reshape the output to 3D
#model.add(Reshape((1, 128)))

# Add a 1D global max pooling layer to reduce the dimensions
model.add(GlobalMaxPooling1D())

# Add a dense layer with activation function
model.add(Dense(64, activation='relu'))

# Add the final dense layer for prediction
model.add(Dense(6, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(A_train, yt, epochs=20, batch_size=32, shuffle=True, validation_split=0.2)

Epoch 1/20
24/24 [==============================] - 5s 135ms/step - loss: 1.6411 - accuracy: 0.3277 - val_loss: 1.4682 - val_accuracy: 0.4188
Epoch 2/20
24/24 [==============================] - 3s 112ms/step - loss: 1.3844 - accuracy: 0.4142 - val_loss: 1.4694 - val_accuracy: 0.4136
Epoch 3/20
24/24 [==============================] - 3s 113ms/step - loss: 1.3936 - accuracy: 0.4260 - val_loss: 1.4602 - val_accuracy: 0.4084
Epoch 4/20
24/24 [==============================] - 3s 114ms/step - loss: 1.2891 - accuracy: 0.4653 - val_loss: 1.4815 - val_accuracy: 0.4241
Epoch 5/20
24/24 [==============================] - 3s 113ms/step - loss: 1.2146 - accuracy: 0.5111 - val_loss: 1.4616 - val_accuracy: 0.4503
Epoch 6/20
24/24 [==============================] - 3s 114ms/step - loss: 1.2139 - accuracy: 0.5321 - val_loss: 1.5215 - val_accuracy: 0.3403
Epoch 7/20
24/24 [==============================] - 3s 115ms/step - loss: 1.1887 - accuracy: 0.5518 - val_loss: 1.5261 - val_accuracy: 0.2565
Epoch 

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score, f1_score

y_pred = model.predict(A_test)
y_pred = np.argmax(y_pred, axis=1)
y_arg = np.argmax(yt2, axis=1)

test_loss, test_acc = model.evaluate(A_test, yt2)

print('Test accuracy:', test_acc)

f1 = f1_score(y_arg, y_pred, average='weighted')
print("F1-score:", f1)

print(confusion_matrix(y_arg, y_pred))

10/10 [==============================] - 0s 45ms/step - loss: 1.5133 - accuracy: 0.3270
Test accuracy: 0.3270440399646759
F1-score: 0.2486808402176895
[[65  0  0  8  0  0]
 [43  0  0  9  0  0]
 [24  0  0 23  0  0]
 [93  0  0 39  0  0]
 [ 1  0  0  0  0  0]
 [12  0  0  1  0  0]]


# Final models?

In [ ]:
# manual random search?

### L1(0.001), 10 epochs, batch_size: 16 

from keras.regularizers import l1, l2
# Define the LSTM model
model = Sequential()

# First LSTM layer with dropout
model.add(LSTM(512, return_sequences=True, input_shape=(A_train.shape[1], A_train.shape[2]),  activity_regularizer=l1(0.001)))
model.add(Dropout(0.5))

# Add a 1D global max pooling layer to reduce the dimensions
model.add(GlobalMaxPooling1D())

# Add a dense layer with activation function
model.add(Dense(64, activation='relu'))

# Add the final dense layer for prediction
model.add(Dense(6, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(A_train, yt, epochs=10, batch_size=16, shuffle=True, validation_split=0.2)

Epoch 1/10
48/48 [==============================] - 6s 89ms/step - loss: 11.9789 - accuracy: 0.3761 - val_loss: 9.9537 - val_accuracy: 0.4188
Epoch 2/10
48/48 [==============================] - 4s 78ms/step - loss: 8.9248 - accuracy: 0.4155 - val_loss: 8.8161 - val_accuracy: 0.4398
Epoch 3/10
48/48 [==============================] - 4s 80ms/step - loss: 7.9141 - accuracy: 0.4469 - val_loss: 8.0775 - val_accuracy: 0.3927
Epoch 4/10
48/48 [==============================] - 4s 80ms/step - loss: 7.2922 - accuracy: 0.5020 - val_loss: 7.6545 - val_accuracy: 0.3822
Epoch 5/10
48/48 [==============================] - 4s 80ms/step - loss: 6.9836 - accuracy: 0.4902 - val_loss: 7.5228 - val_accuracy: 0.1832
Epoch 6/10
48/48 [==============================] - 4s 80ms/step - loss: 6.6691 - accuracy: 0.5085 - val_loss: 7.0507 - val_accuracy: 0.4136
Epoch 7/10
48/48 [==============================] - 4s 80ms/step - loss: 6.4057 - accuracy: 0.5374 - val_loss: 6.8855 - val_accuracy: 0.4241
Epoch 8/10
4

In [ ]:

y_pred = model.predict(A_test)
y_pred = np.argmax(y_pred, axis=1)
y_arg = np.argmax(yt2, axis=1)

test_loss, test_acc = model.evaluate(A_test, yt2)

print('Test accuracy:', test_acc)

f1 = f1_score(y_arg, y_pred, average='weighted')
print("F1-score:", f1)

print(confusion_matrix(y_arg, y_pred))

10/10 [==============================] - 0s 47ms/step - loss: 6.2883 - accuracy: 0.3679
Test accuracy: 0.3679245412349701
F1-score: 0.32236407170424347
[[61  0  1 11  0  0]
 [38  3  1 10  0  0]
 [24  3  5 15  0  0]
 [72  7  5 48  0  0]
 [ 1  0  0  0  0  0]
 [ 7  0  3  3  0  0]]


In [ ]:
### 15 epochs, Dropout 0.2, 128 LSTM units, no regularization

from keras.regularizers import l1, l2
# Define the LSTM model
model = Sequential()

# First LSTM layer with dropout
model.add(LSTM(128, return_sequences=True, input_shape=(A_train.shape[1], A_train.shape[2])))
model.add(Dropout(0.2))

# Add a 1D global max pooling layer to reduce the dimensions
model.add(GlobalMaxPooling1D())

# Add a dense layer with activation function
model.add(Dense(64, activation='relu'))

# Add the final dense layer for prediction
model.add(Dense(6, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(A_train, yt, epochs=15, batch_size=32, shuffle=True, validation_split=0.2)

Epoch 1/15
24/24 [==============================] - 4s 79ms/step - loss: 1.4619 - accuracy: 0.3958 - val_loss: 1.4389 - val_accuracy: 0.4136
Epoch 2/15
24/24 [==============================] - 1s 54ms/step - loss: 1.4143 - accuracy: 0.4102 - val_loss: 1.4257 - val_accuracy: 0.4136
Epoch 3/15
24/24 [==============================] - 1s 52ms/step - loss: 1.3898 - accuracy: 0.4246 - val_loss: 1.4170 - val_accuracy: 0.4136
Epoch 4/15
24/24 [==============================] - 1s 55ms/step - loss: 1.3545 - accuracy: 0.4286 - val_loss: 1.4154 - val_accuracy: 0.4136
Epoch 5/15
24/24 [==============================] - 1s 56ms/step - loss: 1.3335 - accuracy: 0.4469 - val_loss: 1.4532 - val_accuracy: 0.4293
Epoch 6/15
24/24 [==============================] - 1s 57ms/step - loss: 1.3278 - accuracy: 0.4784 - val_loss: 1.4276 - val_accuracy: 0.4084
Epoch 7/15
24/24 [==============================] - 2s 75ms/step - loss: 1.3233 - accuracy: 0.4522 - val_loss: 1.5164 - val_accuracy: 0.2775
Epoch 8/15
24

In [ ]:

y_pred = model.predict(A_test)
y_pred = np.argmax(y_pred, axis=1)
y_arg = np.argmax(yt2, axis=1)

test_loss, test_acc = model.evaluate(A_test, yt2)

print('Test accuracy:', test_acc)

f1 = f1_score(y_arg, y_pred, average='weighted')
print("F1-score:", f1)

print(confusion_matrix(y_arg, y_pred))

10/10 [==============================] - 0s 28ms/step - loss: 1.3998 - accuracy: 0.4119
Test accuracy: 0.41194969415664673
F1-score: 0.2506894124144798
[[  1   2   0  70   0   0]
 [  3   0   0  49   0   0]
 [  0   1   0  46   0   0]
 [  0   2   0 130   0   0]
 [  0   0   0   1   0   0]
 [  0   0   0  13   0   0]]


In [ ]:
### 10 epochs, Dropout 0.5, 3 dense layers, 512 LSTM units, L2 regularization

from keras.regularizers import l1, l2
# Define the LSTM model
model = Sequential()

# First LSTM layer with dropout
model.add(LSTM(512, return_sequences=True, input_shape=(A_train.shape[1], A_train.shape[2]),activity_regularizer=l1(0.001)))
model.add(Dropout(0.5))

# Add a 1D global max pooling layer to reduce the dimensions
model.add(GlobalMaxPooling1D())

# Add a dense layer with activation function
model.add(Dense(128, activation='relu'))

model.add(Dense(64, activation='relu'))

# Add the final dense layer for prediction
model.add(Dense(6, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(A_train, yt, epochs=10, batch_size=32, shuffle=True, validation_split=0.2)

Epoch 1/10
24/24 [==============================] - 5s 140ms/step - loss: 12.6852 - accuracy: 0.3211 - val_loss: 10.0676 - val_accuracy: 0.3298
Epoch 2/10
24/24 [==============================] - 3s 117ms/step - loss: 8.5899 - accuracy: 0.4273 - val_loss: 8.3163 - val_accuracy: 0.4136
Epoch 3/10
24/24 [==============================] - 3s 116ms/step - loss: 7.4475 - accuracy: 0.4364 - val_loss: 7.6817 - val_accuracy: 0.3717
Epoch 4/10
24/24 [==============================] - 3s 117ms/step - loss: 6.9081 - accuracy: 0.4889 - val_loss: 7.3520 - val_accuracy: 0.3089
Epoch 5/10
24/24 [==============================] - 3s 117ms/step - loss: 6.5229 - accuracy: 0.5229 - val_loss: 7.0320 - val_accuracy: 0.4241
Epoch 6/10
24/24 [==============================] - 3s 117ms/step - loss: 6.2737 - accuracy: 0.5491 - val_loss: 6.9177 - val_accuracy: 0.2461
Epoch 7/10
24/24 [==============================] - 3s 117ms/step - loss: 6.0523 - accuracy: 0.5334 - val_loss: 6.6560 - val_accuracy: 0.4293
Epoc

In [ ]:

y_pred = model.predict(A_test)
y_pred = np.argmax(y_pred, axis=1)
y_arg = np.argmax(yt2, axis=1)

test_loss, test_acc = model.evaluate(A_test, yt2)

print('Test accuracy:', test_acc)

f1 = f1_score(y_arg, y_pred, average='weighted')
print("F1-score:", f1)

print(confusion_matrix(y_arg, y_pred))

10/10 [==============================] - 0s 46ms/step - loss: 6.2921 - accuracy: 0.3899
Test accuracy: 0.3899371027946472
F1-score: 0.324298563641427
[[53  1  0 19  0  0]
 [33  2  1 16  0  0]
 [19  0  0 28  0  0]
 [56  6  1 69  0  0]
 [ 1  0  0  0  0  0]
 [ 6  2  0  5  0  0]]


In [ ]:
### BatchSize 16, Dropout 0.5, 1 dense layers, 512 LSTM units, L1 regularization


# Define the LSTM model
model = Sequential()

# First LSTM layer with dropout
model.add(LSTM(512, return_sequences=True, input_shape=(A_train.shape[1], A_train.shape[2]),activity_regularizer=l1(0.001)))
model.add(Dropout(0.5))

# Add a 1D global max pooling layer to reduce the dimensions
model.add(GlobalMaxPooling1D())


# Add the final dense layer for prediction
model.add(Dense(6, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(A_train, yt, epochs=15, batch_size=16, shuffle=True, validation_split=0.2)

Epoch 1/15
48/48 [==============================] - 6s 93ms/step - loss: 12.8569 - accuracy: 0.3250 - val_loss: 10.9561 - val_accuracy: 0.4293
Epoch 2/15
48/48 [==============================] - 4s 79ms/step - loss: 9.7813 - accuracy: 0.4233 - val_loss: 9.6999 - val_accuracy: 0.3874
Epoch 3/15
48/48 [==============================] - 4s 79ms/step - loss: 8.7787 - accuracy: 0.4889 - val_loss: 9.0645 - val_accuracy: 0.3822
Epoch 4/15
48/48 [==============================] - 4s 79ms/step - loss: 8.1698 - accuracy: 0.5596 - val_loss: 8.6071 - val_accuracy: 0.3351
Epoch 5/15
48/48 [==============================] - 4s 79ms/step - loss: 7.6599 - accuracy: 0.6094 - val_loss: 8.1913 - val_accuracy: 0.4084
Epoch 6/15
48/48 [==============================] - 4s 81ms/step - loss: 7.3442 - accuracy: 0.5937 - val_loss: 7.9929 - val_accuracy: 0.2670
Epoch 7/15
48/48 [==============================] - 4s 81ms/step - loss: 7.0062 - accuracy: 0.6422 - val_loss: 7.6128 - val_accuracy: 0.4188
Epoch 8/15


In [ ]:

y_pred = model.predict(A_test)
y_pred = np.argmax(y_pred, axis=1)
y_arg = np.argmax(yt2, axis=1)

test_loss, test_acc = model.evaluate(A_test, yt2)

print('Test accuracy:', test_acc)

f1 = f1_score(y_arg, y_pred, average='weighted')
print("F1-score:", f1)

print(confusion_matrix(y_arg, y_pred))

10/10 [==============================] - 0s 46ms/step - loss: 6.3776 - accuracy: 0.3396
Test accuracy: 0.3396226465702057
F1-score: 0.3202238712908668
[[39 27  1  6  0  0]
 [21 21  1  9  0  0]
 [17 14  0 16  0  0]
 [42 41  1 48  0  0]
 [ 1  0  0  0  0  0]
 [ 3  6  1  3  0  0]]


In [ ]:
#10 epochs, Dropout 0.5, 2 dense layers with 32 units, 128 LSTM units, no regularization


# Define the LSTM model
model = Sequential()

# First LSTM layer with dropout
model.add(LSTM(128, return_sequences=True, input_shape=(A_train.shape[1], A_train.shape[2]),))
model.add(Dropout(0.5))

# Add a 1D global max pooling layer to reduce the dimensions
model.add(GlobalMaxPooling1D())
model.add(Dense(32, activation='relu'))

# Add the final dense layer for prediction
model.add(Dense(6, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(A_train, yt, epochs=10, batch_size=32, shuffle=True, validation_split=0.2)

Epoch 1/10
24/24 [==============================] - 3s 77ms/step - loss: 1.7279 - accuracy: 0.3303 - val_loss: 1.5078 - val_accuracy: 0.4136
Epoch 2/10
24/24 [==============================] - 1s 56ms/step - loss: 1.4072 - accuracy: 0.4286 - val_loss: 1.4887 - val_accuracy: 0.4136
Epoch 3/10
24/24 [==============================] - 1s 54ms/step - loss: 1.3699 - accuracy: 0.4142 - val_loss: 1.5021 - val_accuracy: 0.3770
Epoch 4/10
24/24 [==============================] - 1s 52ms/step - loss: 1.3611 - accuracy: 0.4377 - val_loss: 1.4917 - val_accuracy: 0.3927
Epoch 5/10
24/24 [==============================] - 1s 52ms/step - loss: 1.3355 - accuracy: 0.4246 - val_loss: 1.5081 - val_accuracy: 0.3613
Epoch 6/10
24/24 [==============================] - 1s 53ms/step - loss: 1.3117 - accuracy: 0.4666 - val_loss: 1.5249 - val_accuracy: 0.2565
Epoch 7/10
24/24 [==============================] - 1s 55ms/step - loss: 1.3157 - accuracy: 0.4535 - val_loss: 1.4810 - val_accuracy: 0.4136
Epoch 8/10
24

In [ ]:

y_pred = model.predict(A_test)
y_pred = np.argmax(y_pred, axis=1)
y_arg = np.argmax(yt2, axis=1)

test_loss, test_acc = model.evaluate(A_test, yt2)

print('Test accuracy:', test_acc)

f1 = f1_score(y_arg, y_pred, average='weighted')
print("F1-score:", f1)

print(confusion_matrix(y_arg, y_pred))

10/10 [==============================] - 0s 27ms/step - loss: 1.4768 - accuracy: 0.4182
Test accuracy: 0.4182389974594116
F1-score: 0.3424354640668033
[[44  1  0 28  0  0]
 [26  2  0 24  0  0]
 [16  2  0 29  0  0]
 [45  0  0 87  0  0]
 [ 1  0  0  0  0  0]
 [ 4  1  0  8  0  0]]


In [ ]:
### All best parameters:

# Define the LSTM model
model = Sequential()

# First LSTM layer with dropout
model.add(LSTM(512, return_sequences=True, input_shape=(A_train.shape[1], A_train.shape[2]),activity_regularizer=l1(0.001)))
model.add(Dropout(0.5))

# Add a 1D global max pooling layer to reduce the dimensions
model.add(GlobalMaxPooling1D())

model.add(Dense(64, activation='relu'))

# Add the final dense layer for prediction
model.add(Dense(6, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(A_train, yt, epochs=10, batch_size=32, shuffle=True, validation_split=0.2)

Epoch 1/10
24/24 [==============================] - 5s 138ms/step - loss: 13.5170 - accuracy: 0.3421 - val_loss: 11.0136 - val_accuracy: 0.4136
Epoch 2/10
24/24 [==============================] - 3s 116ms/step - loss: 9.5429 - accuracy: 0.4456 - val_loss: 9.3847 - val_accuracy: 0.2199
Epoch 3/10
24/24 [==============================] - 3s 117ms/step - loss: 8.2827 - accuracy: 0.5020 - val_loss: 8.6636 - val_accuracy: 0.3194
Epoch 4/10
24/24 [==============================] - 3s 117ms/step - loss: 7.6921 - accuracy: 0.5308 - val_loss: 8.1108 - val_accuracy: 0.4188
Epoch 5/10
24/24 [==============================] - 3s 117ms/step - loss: 7.2640 - accuracy: 0.5413 - val_loss: 7.8256 - val_accuracy: 0.3455
Epoch 6/10
24/24 [==============================] - 3s 117ms/step - loss: 6.9014 - accuracy: 0.6173 - val_loss: 7.5626 - val_accuracy: 0.3351
Epoch 7/10
24/24 [==============================] - 3s 117ms/step - loss: 6.6134 - accuracy: 0.6461 - val_loss: 7.3431 - val_accuracy: 0.2670
Epoc

In [ ]:
y_pred = model.predict(A_test)
y_pred = np.argmax(y_pred, axis=1)
y_arg = np.argmax(yt2, axis=1)

test_loss, test_acc = model.evaluate(A_test, yt2)

print('Test accuracy:', test_acc)

f1 = f1_score(y_arg, y_pred, average='weighted')
print("F1-score:", f1)

print(confusion_matrix(y_arg, y_pred))

10/10 [==============================] - 0s 47ms/step - loss: 6.7133 - accuracy: 0.4088
Test accuracy: 0.4088050425052643
F1-score: 0.3761431148442996
[[22 38  0 13  0  0]
 [ 4 31  0 17  0  0]
 [12 16  0 19  0  0]
 [15 40  0 77  0  0]
 [ 1  0  0  0  0  0]
 [ 2  5  0  6  0  0]]


In [ ]:
### All best parameters again:

# Define the LSTM model
model = Sequential()

# First LSTM layer with dropout
model.add(LSTM(512, return_sequences=True, input_shape=(A_train.shape[1], A_train.shape[2]),activity_regularizer=l1(0.001)))
model.add(Dropout(0.5))

# Add a 1D global max pooling layer to reduce the dimensions
model.add(GlobalMaxPooling1D())

model.add(Dense(64, activation='relu'))

# Add the final dense layer for prediction
model.add(Dense(6, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(A_train, yt, epochs=10, batch_size=32, shuffle=True, validation_split=0.2)

Epoch 1/10
24/24 [==============================] - 5s 141ms/step - loss: 13.5010 - accuracy: 0.3486 - val_loss: 11.2364 - val_accuracy: 0.3822
Epoch 2/10
24/24 [==============================] - 3s 116ms/step - loss: 9.9038 - accuracy: 0.3801 - val_loss: 9.5441 - val_accuracy: 0.2827
Epoch 3/10
24/24 [==============================] - 3s 117ms/step - loss: 8.5170 - accuracy: 0.4849 - val_loss: 8.7390 - val_accuracy: 0.3979
Epoch 4/10
24/24 [==============================] - 3s 118ms/step - loss: 7.8416 - accuracy: 0.5046 - val_loss: 8.2859 - val_accuracy: 0.2618
Epoch 5/10
24/24 [==============================] - 3s 118ms/step - loss: 7.3076 - accuracy: 0.5845 - val_loss: 7.8375 - val_accuracy: 0.3874
Epoch 6/10
24/24 [==============================] - 3s 118ms/step - loss: 6.9522 - accuracy: 0.5885 - val_loss: 7.6396 - val_accuracy: 0.2880
Epoch 7/10
24/24 [==============================] - 3s 119ms/step - loss: 6.6153 - accuracy: 0.6540 - val_loss: 7.4112 - val_accuracy: 0.2723
Epoc

In [ ]:
y_pred = model.predict(A_test)
y_pred = np.argmax(y_pred, axis=1)
y_arg = np.argmax(yt2, axis=1)

test_loss, test_acc = model.evaluate(A_test, yt2)

print('Test accuracy:', test_acc)

f1 = f1_score(y_arg, y_pred, average='weighted')
print("F1-score:", f1)

print(confusion_matrix(y_arg, y_pred))

10/10 [==============================] - 0s 46ms/step - loss: 6.6534 - accuracy: 0.4182
Test accuracy: 0.4182389974594116
F1-score: 0.37087219947328764
[[23  7  7 36  0  0]
 [12  5  7 28  0  0]
 [ 8  4  6 29  0  0]
 [16  3 14 99  0  0]
 [ 0  1  0  0  0  0]
 [ 1  3  2  7  0  0]]


# Random trials

In [ ]:
# random trials

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Flatten, Reshape
from keras.regularizers import l1, l2

# Define the LSTM model
model = Sequential()

# First LSTM layer with dropout and L2 regularization
model.add(LSTM(512, return_sequences=True, input_shape=(A_train.shape[1], A_train.shape[2]), activity_regularizer=l2(0.001)))
model.add(Dropout(0.4))

# Second LSTM layer with dropout and L2 regularization
model.add(LSTM(128, return_sequences=False, activity_regularizer=l2(0.001)))
model.add(Dropout(0.4))

model.add(Reshape((1, 128)))
model.add(GlobalMaxPooling1D())

# Add a dense layer with activation function
model.add(Dense(64, activation='relu'))

# Add the final dense layer for prediction
model.add(Dense(6, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(A_train, yt, epochs=15, batch_size=32, shuffle=True, validation_split=0.2)

InternalError: ignored

In [ ]:
score = model.evaluate(A_test, yt2, verbose=0)
print("Test Loss: ", score[0])
print("Test Accuracy: ", score[1])

Test Loss:  2.082176923751831
Test Accuracy:  0.4150943458080292


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score, f1_score


y_pred = model.predict(A_test)
y_pred = np.argmax(y_pred, axis=1)
y_arg = np.argmax(yt2, axis=1)

test_loss, test_acc = model.evaluate(A_test, yt2)

print('Test accuracy:', test_acc)

f1 = f1_score(y_arg, y_pred, average='micro')
print("F1-score:", f1)

print(confusion_matrix(y_arg, y_pred))

10/10 [==============================] - 1s 63ms/step - loss: 2.0822 - accuracy: 0.4151
Test accuracy: 0.4150943458080292
F1-score: 0.41509433962264153
[[  0   0   0  73   0   0]
 [  0   0   0  52   0   0]
 [  0   0   0  47   0   0]
 [  0   0   0 132   0   0]
 [  0   0   0   1   0   0]
 [  0   0   0  13   0   0]]


In [ ]:
# Initialize the LSTM model
model = Sequential()

# Add an LSTM layer with 128 units
model.add(LSTM(128, input_shape=(A_train.shape[1], A_train.shape[2])))

# Add dropout regularization to prevent overfitting
model.add(Dropout(0.5))

# Add a dense layer with 6 units and a softmax activation function
model.add(Dense(6, activation='softmax'))

# Compile the model with categorical crossentropy loss and the Adam optimizer
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model on the training data for 20 epochs with a batch size of 32
model.fit(A_train, yt, epochs=20, batch_size=32,validation_split=0.2)


Epoch 1/20
24/24 [==============================] - 4s 105ms/step - loss: 1.6354 - accuracy: 0.3932 - val_loss: 1.4828 - val_accuracy: 0.4136
Epoch 2/20
24/24 [==============================] - 2s 72ms/step - loss: 1.4710 - accuracy: 0.3906 - val_loss: 1.4522 - val_accuracy: 0.4136
Epoch 3/20
24/24 [==============================] - 1s 59ms/step - loss: 1.4587 - accuracy: 0.3932 - val_loss: 1.4539 - val_accuracy: 0.4136
Epoch 4/20
24/24 [==============================] - 1s 58ms/step - loss: 1.4604 - accuracy: 0.4050 - val_loss: 1.4429 - val_accuracy: 0.4136
Epoch 5/20
24/24 [==============================] - 2s 63ms/step - loss: 1.4727 - accuracy: 0.4050 - val_loss: 1.4500 - val_accuracy: 0.4136
Epoch 6/20
24/24 [==============================] - 2s 74ms/step - loss: 1.4469 - accuracy: 0.4115 - val_loss: 1.4400 - val_accuracy: 0.4136
Epoch 7/20
24/24 [==============================] - 2s 71ms/step - loss: 1.4517 - accuracy: 0.4076 - val_loss: 1.4454 - val_accuracy: 0.4136
Epoch 8/20
2

In [ ]:
y_pred = model.predict(A_test)
y_pred = np.argmax(y_pred, axis=1)
y_arg = np.argmax(yt2, axis=1)

test_loss, test_acc = model.evaluate(A_test, yt2)

print('Test accuracy:', test_acc)

f1 = f1_score(y_arg, y_pred, average='micro')
print("F1-score:", f1)

print(confusion_matrix(y_arg, y_pred))

10/10 [==============================] - 1s 44ms/step - loss: 1.4404 - accuracy: 0.4151
Test accuracy: 0.4150943458080292
F1-score: 0.41509433962264153
[[  0   0   0  73   0   0]
 [  0   0   0  52   0   0]
 [  0   0   0  47   0   0]
 [  0   0   0 132   0   0]
 [  0   0   0   1   0   0]
 [  0   0   0  13   0   0]]


In [ ]:
from keras.optimizers import Adam
opt = Adam(learning_rate=0.00001)

In [ ]:
def lstm_mod_1():
    model = Sequential()
    model.add(LSTM(512, return_sequences=True, input_shape=(A_train.shape[1],A_train.shape[2])))
    model.add(LSTM(256, return_sequences=False))
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dense(6))
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

    return model

In [ ]:
test_mod = lstm_mod_1()
test_mod.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 2488, 512)         1329152   
                                                                 
 lstm_1 (LSTM)               (None, 256)               787456    
                                                                 
 dense (Dense)               (None, 512)               131584    
                                                                 
 activation (Activation)     (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 6)                 3078      
                                                                 
 activation_1 (Activation)   (None, 6)                 0         
                                                                 
Total params: 2,251,270
Trainable params: 2,251,270
Non-

In [ ]:
# Trial
hist = test_mod.fit(A_train, yt, batch_size=34, epochs=15, verbose=1, shuffle=False,validation_split=0.25)

Epoch 1/15
22/22 [==============================] - 24s 713ms/step - loss: 1.7913 - accuracy: 0.3706 - val_loss: 1.7904 - val_accuracy: 0.4142
Epoch 2/15
22/22 [==============================] - 13s 607ms/step - loss: 1.7891 - accuracy: 0.4098 - val_loss: 1.7880 - val_accuracy: 0.4142
Epoch 3/15
22/22 [==============================] - 16s 725ms/step - loss: 1.7858 - accuracy: 0.4098 - val_loss: 1.7839 - val_accuracy: 0.4142
Epoch 4/15
22/22 [==============================] - 14s 650ms/step - loss: 1.7794 - accuracy: 0.4098 - val_loss: 1.7745 - val_accuracy: 0.4142
Epoch 5/15
22/22 [==============================] - 14s 630ms/step - loss: 1.7591 - accuracy: 0.4098 - val_loss: 1.7317 - val_accuracy: 0.4142
Epoch 6/15
22/22 [==============================] - 14s 647ms/step - loss: 1.6171 - accuracy: 0.4098 - val_loss: 1.5259 - val_accuracy: 0.4142
Epoch 7/15
22/22 [==============================] - 14s 655ms/step - loss: 1.4750 - accuracy: 0.4098 - val_loss: 1.5071 - val_accuracy: 0.4142

In [ ]:
yt2 = label_binarize(y_test, classes=['angry', 'happy', 'sad', 'neutral', 'other', 'X'])
A_test=A_test.reshape(A_test.shape[0],A_train.shape[1],136)

test_loss, test_acc = test_mod.evaluate(A_test, yt2)


10/10 [==============================] - 2s 192ms/step - loss: 1.4394 - accuracy: 0.4151


In [ ]:
print('Test accuracy:', test_acc)

Test accuracy: 0.4150943458080292


In [ ]:
hist2 = test_mod.fit(A_train, yt, batch_size=34, epochs=15, verbose=1, shuffle=True,validation_split=0.25)

Epoch 1/15
22/22 [==============================] - 14s 640ms/step - loss: 1.4645 - accuracy: 0.4000 - val_loss: 1.6703 - val_accuracy: 0.4142
Epoch 2/15
22/22 [==============================] - 14s 644ms/step - loss: 1.4649 - accuracy: 0.4098 - val_loss: 1.4354 - val_accuracy: 0.4142
Epoch 3/15
22/22 [==============================] - 14s 647ms/step - loss: 1.4474 - accuracy: 0.4098 - val_loss: 1.5845 - val_accuracy: 0.2176
Epoch 4/15
22/22 [==============================] - 15s 675ms/step - loss: 1.4568 - accuracy: 0.3888 - val_loss: 1.4571 - val_accuracy: 0.4142
Epoch 5/15
22/22 [==============================] - 14s 662ms/step - loss: 1.4505 - accuracy: 0.4098 - val_loss: 1.4948 - val_accuracy: 0.4142
Epoch 6/15
22/22 [==============================] - 14s 657ms/step - loss: 1.4452 - accuracy: 0.4098 - val_loss: 1.4694 - val_accuracy: 0.4142
Epoch 7/15
22/22 [==============================] - 14s 658ms/step - loss: 1.4468 - accuracy: 0.4098 - val_loss: 1.4825 - val_accuracy: 0.4142

KeyboardInterrupt: ignored

In [ ]:
# Trial #2
from keras.optimizers import SGD
opt = SGD(learning_rate=0.1)

def lstm_mod_2():
    model = Sequential()
    model.add(LSTM(512, return_sequences=True, input_shape=(A_train.shape[1],A_train.shape[2])))
    model.add(LSTM(256, return_sequences=False))
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dense(256))
    model.add(Activation('sigmoid'))
    model.add(Dense(6))
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    return model

In [ ]:
test_mod = lstm_mod_2()
test_mod.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_6 (LSTM)               (None, 2488, 512)         1329152   
                                                                 
 lstm_7 (LSTM)               (None, 256)               787456    
                                                                 
 dense_6 (Dense)             (None, 512)               131584    
                                                                 
 activation_6 (Activation)   (None, 512)               0         
                                                                 
 dense_7 (Dense)             (None, 256)               131328    
                                                                 
 activation_7 (Activation)   (None, 256)               0         
                                                                 
 dense_8 (Dense)             (None, 6)                

In [ ]:
hist = test_mod.fit(A_train, yt, batch_size=53, epochs=15, verbose=1, shuffle=True,validation_split=0.25)

Epoch 1/15
18/18 [==============================] - 15s 715ms/step - loss: 1.4980 - accuracy: 0.4109
Epoch 2/15
18/18 [==============================] - 13s 741ms/step - loss: 1.4480 - accuracy: 0.4109
Epoch 3/15
18/18 [==============================] - 13s 733ms/step - loss: 1.4408 - accuracy: 0.4109
Epoch 4/15
18/18 [==============================] - 13s 740ms/step - loss: 1.4403 - accuracy: 0.4109
Epoch 5/15
 9/18 [==============>...............] - ETA: 6s - loss: 1.4434 - accuracy: 0.4109

KeyboardInterrupt: ignored

In [ ]:
As_train = 0

In [ ]:
def lstm_mod_3():
    model = Sequential()
    model.add(LSTM(512, return_sequences=True, input_shape=(5287,64)))
    model.add(LSTM(256, return_sequences=False))
    
    model.add(Dense(512))
    model.add(Activation('relu'))

    model.add(Dense(256))
    model.add(Activation('relu'))

    model.add(Dense(128))
    model.add(Activation('relu'))

    model.add(Dense(6))
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

    return model

In [ ]:
testmod = lstm_mod_3()
testmod.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_9 (LSTM)               (None, 5287, 512)         1181696   
                                                                 
 lstm_10 (LSTM)              (None, 256)               787456    
                                                                 
 dense_8 (Dense)             (None, 512)               131584    
                                                                 
 activation_8 (Activation)   (None, 512)               0         
                                                                 
 dense_9 (Dense)             (None, 256)               131328    
                                                                 
 activation_9 (Activation)   (None, 256)               0         
                                                                 
 dense_10 (Dense)            (None, 128)              

In [ ]:
hist = testmod.fit(A_train, yt, batch_size=53, epochs=10, verbose=1, shuffle=False,validation_split=0.2)

Epoch 1/10
15/15 [==============================] - 25s 1s/step - loss: 1.7406 - accuracy: 0.3028 - val_loss: 1.5060 - val_accuracy: 0.4136
Epoch 2/10
15/15 [==============================] - 22s 1s/step - loss: 1.4962 - accuracy: 0.3879 - val_loss: 1.4648 - val_accuracy: 0.4136
Epoch 3/10
15/15 [==============================] - 23s 2s/step - loss: 1.4709 - accuracy: 0.4102 - val_loss: 1.4741 - val_accuracy: 0.4136
Epoch 4/10
15/15 [==============================] - 23s 2s/step - loss: 1.4660 - accuracy: 0.4024 - val_loss: 1.4436 - val_accuracy: 0.4136
Epoch 5/10
15/15 [==============================] - 22s 1s/step - loss: 1.4613 - accuracy: 0.4063 - val_loss: 1.4426 - val_accuracy: 0.4136
Epoch 6/10
15/15 [==============================] - 22s 1s/step - loss: 1.4619 - accuracy: 0.4089 - val_loss: 1.4312 - val_accuracy: 0.4136
Epoch 7/10
15/15 [==============================] - 22s 1s/step - loss: 1.4559 - accuracy: 0.4102 - val_loss: 1.4285 - val_accuracy: 0.4136
Epoch 8/10
15/15 [==

In [ ]:
### PARAMETERS tuning with RandomSearchCV

from keras.wrappers.scikit_learn import KerasClassifier
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.regularizers import l1_l2
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import f1_score, accuracy_score
import numpy as np

# Define the model architecture
def create_model(units_1, units_2, dropout_1, dropout_2, l1, l2):
    model = Sequential()
    model.add(LSTM(units_1, input_shape=(A_train.shape[1],A_train.shape[2]), return_sequences=True,
                   activity_regularizer=l1_l2(l1=l1, l2=l2)))
    model.add(Dropout(dropout_1))
    model.add(LSTM(units_2, activity_regularizer=l1_l2(l1=l1, l2=l2)))
    model.add(Dropout(dropout_2))
    model.add(Dense(6, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# Define the parameter distribution for the random search
param_dist = {
    'units_1': [256, 128, 512],
    'units_2': [64, 128, 256],
    'dropout_1': [0.2, 0.5, 0.8],
    'dropout_2': [0.2, 0.5, 0.8],
    'l1': [0, 0.001, 0.0001],
    'l2': [0, 0.001, 0.0001]
}

model = KerasClassifier(build_fn=create_model, epochs=15, batch_size=32, verbose=0)

# Perform the random search
random_search = RandomizedSearchCV(estimator=model, param_distributions=param_dist, n_iter=10, cv=5, 
                                   scoring={'f1_micro': 'f1_micro', 'f1_macro': 'f1_macro', 'accuracy': 'accuracy'}, 
                                   refit='f1_micro')
random_search.fit(A_train, yt, validation_split=0.2)

# Print the best parameters and the best f1_micro score
print("Best parameters found: ", random_search.best_params_)
print("Best f1_micro score: ", random_search.best_score_)

<ipython-input-15-711920378809>:33: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn=create_model, epochs=15, batch_size=32, verbose=0)


6/6 [==============================] - 1s 41ms/step


/usr/local/lib/python3.8/dist-packages/sklearn/model_selection/_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/sklearn/model_selection/_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_scorer.py", line 103, in __call__
    score = scorer._score(cached_call, estimator, *args, **kwargs)
  File "/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_scorer.py", line 264, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py", line 1123, in f1_score
    return fbeta_score(
  File "/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py", line 1261, in fbeta_score
    _, _, f, _ = precision_recall

6/6 [==============================] - 1s 41ms/step


/usr/local/lib/python3.8/dist-packages/sklearn/model_selection/_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/sklearn/model_selection/_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_scorer.py", line 103, in __call__
    score = scorer._score(cached_call, estimator, *args, **kwargs)
  File "/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_scorer.py", line 264, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py", line 1123, in f1_score
    return fbeta_score(
  File "/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py", line 1261, in fbeta_score
    _, _, f, _ = precision_recall

6/6 [==============================] - 1s 41ms/step


/usr/local/lib/python3.8/dist-packages/sklearn/model_selection/_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/sklearn/model_selection/_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_scorer.py", line 103, in __call__
    score = scorer._score(cached_call, estimator, *args, **kwargs)
  File "/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_scorer.py", line 264, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py", line 1123, in f1_score
    return fbeta_score(
  File "/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py", line 1261, in fbeta_score
    _, _, f, _ = precision_recall

6/6 [==============================] - 1s 42ms/step


/usr/local/lib/python3.8/dist-packages/sklearn/model_selection/_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/sklearn/model_selection/_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_scorer.py", line 103, in __call__
    score = scorer._score(cached_call, estimator, *args, **kwargs)
  File "/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_scorer.py", line 264, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py", line 1123, in f1_score
    return fbeta_score(
  File "/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py", line 1261, in fbeta_score
    _, _, f, _ = precision_recall

KeyboardInterrupt: ignored